# Set env:

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
from scipy.stats import norm, linregress
import math
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import display, HTML
from tqdm import tqdm 
pd.set_option('display.width', 1000)
sns.set_style("white")

# Define functions:

REFERENCE SAMPLE: Function to create simuation_matrix. Creates plots of Point of subjective simultaneity (PSS) and accuracy trajectories

In [ ]:
def reference_matrix(simulation_df, pred_pss_values, pred_accuracies, drop = True, plots = False):
    
    # Recode trial numbers: Make trials continuous from 1-30
    simulation_df['Recode_Trial'] = np.where(simulation_df['Staircase_name'].str.contains('400'),
                                             simulation_df['Trial'], 
                                             simulation_df['Trial'] + 15)
    
    
    if plots:   
        colors = plt.cm.viridis(np.linspace(0, 1, len(pred_accuracies)))  # Color map for accuracy levels

        # Create a grid for the subplots
        n_rows = 4
        n_cols = 2
        n_iterations = 100
        fig, axs = plt.subplots(n_rows, n_cols, figsize=(12,12), dpi=300)
        axs = axs.ravel()  # Flatten the 2D array of axes to 1D for easy iteration
        plt.rcParams.update({'font.size': 8})
        for i, pss_value in enumerate(pred_pss_values):
            ax = axs[i]
            df_pss = simulation_df[simulation_df['PSS_value'] == pss_value]

            # Set title for the subplot
            ax.set_title(f'{pss_value} ms', fontsize=16,  loc='left')

            # Loop through each accuracy value
            for accuracy_idx, accuracy in enumerate(pred_accuracies):
                # Filter data by accuracy and group by Staircase_name and Trial
                df_accuracy = df_pss[df_pss['Accuracy'] == accuracy]
                mean_stim_values = df_accuracy.groupby(['Staircase_name', 'Trial'])['Current Delay'].mean()
                std_stim_values = df_accuracy.groupby(['Staircase_name', 'Trial'])['Current Delay'].std()

                # Plot each staircase separately
                for staircase_name, group in mean_stim_values.groupby(level=0):
                    # Compute 95% confidence intervals
                    trial_numbers = group.index.get_level_values(1)
                    #ci_95 = 1.96 * std_stim_values.loc[staircase_name] / np.sqrt(n_iterations)


                    x_vals = group.index.get_level_values(1).to_numpy()
                    y_vals = group.values.astype(float).flatten()  # Ensure 1D
                    ci_vals = std_stim_values.loc[staircase_name].values.astype(float).flatten() * 1.96 / np.sqrt(n_iterations)
                    ax.plot(x_vals, y_vals, 
                            label=f'{staircase_name} - Accuracy {accuracy*100:.0f}%', 
                            color=colors[accuracy_idx], 
                            marker='o')

                    ax.fill_between(x_vals, 
                                    y_vals - ci_vals, 
                                    y_vals + ci_vals, 
                                    color=colors[accuracy_idx], 
                                    alpha=0.2)

                    # Plot the mean stimulus values and confidence intervals
                    #ax.plot(group.index.get_level_values(1), group.values, 
                    #        label=f'{staircase_name} - Accuracy {accuracy*100:.0f}%', 
                    #        color=colors[accuracy_idx], 
                    #        marker='o')

                    #ax.fill_between(group.index.get_level_values(1), 
                    #                group.values - ci_95, 
                    #                group.values + ci_95, 
                    #                color=colors[accuracy_idx], 
                    #                alpha=0.2)

            # Labeling the subplot
            ax.set_xlabel('Trial')
            ax.set_ylabel('Delay (ms)')
            ax.set_ylim(0, 800)  # Set y-axis limit between 0 and 800
            #ax.legend(loc='upper right')
            ax.axhspan((pss_value-1), (pss_value +1), color='red', alpha=1.0, linestyle = '--', zorder=10)

            # Adjust layout for better spacing between subplots
            plt.tight_layout()

            
            save_path = '/PATH/Reference_trajectories_100_400.png'
            
        plt.savefig(save_path) 
        plt.show()
        print(f"Saved to {save_path}")
    
    if drop: 
        #Remove the first two trials from each staircase
        #simulation_df = simulation_df[~simulation_df['Recode_Trial'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])] #last 5 trials only
        simulation_df = simulation_df[~simulation_df['Recode_Trial'].isin([1, 2, 3, 4, 5, 16, 17, 18, 19, 20])] #trials 5-15

    
    # Create a pivot table to calculate mean delay
    pivot_df = simulation_df.pivot_table(
        index=['PSS_value', 'Recode_Trial'],  # Depth: PSS_value; Rows: Trial number
        columns='Accuracy',                  # Columns: Accuracy values
        values='Current Delay',              # Values: Mean Delay
        aggfunc='mean'                       # Aggregate function: mean
    )
    
    # Only use PSS = 100-400; exclude PSS isin([50, 450, 500, 550, 600]
    pivot_df = pivot_df.loc[~pivot_df.index.get_level_values('PSS_value').isin([50, 450, 500, 550, 600])]
    
    # Fill missing values
    pivot_df = pivot_df.fillna(0)
    
    # Prepare labels for heatmaps
    trial_labels = [f"Trial {i}" for i in sorted(simulation_df['Recode_Trial'].unique())]
    accuracy_labels = [f"Acc {round(acc, 2)}" for acc in sorted(simulation_df['Accuracy'].unique())]
    pss_values = sorted(pivot_df.index.get_level_values('PSS_value').unique())
    #pss_values = sorted(simulation_df[~simulation_df['PSS_value'].isin([50, 450, 500, 550, 600])

    return pivot_df


REFERENCE SAMPLE: Function to compute response counts

In [ ]:
def compute_response_counts(participants_df, participant_id, i):
    # Count the occurrences of each response code per Staircase_name
    response_counts = participants_df.groupby(['Staircase_name', 'Response_code']).size().unstack(fill_value=0)
    
    # Rename columns for clarity
    response_counts = response_counts.rename(columns={-1: "Before_Response", 
                                                       0: "Same_Response", 
                                                       1: "After_Response"})
    
    # Reset index for merging or visualization
    response_counts = response_counts.reset_index()
    
    # Add participant_id column
    response_counts['participant_id'] = f"{participant_id}_{i}"
    
    return response_counts

REFERENCE SAMPLE: Function to calculate how many participants had >5 or <15 button presses

In [ ]:
def get_participants_outside_range_melted(df, staircase_name):
    """
    Returns a set of participant IDs whose any Count value for the given staircase_name is outside [5, 15]
    """
    filtered = df[df['Modified_Staircase_name'] == staircase_name]
    
    # Find rows where Count is outside the range [5, 15]
    outside_range = filtered[(filtered['Count'] < 5) | (filtered['Count'] > 15)]
    
    # Return unique participant_ids
    return set(outside_range['participant_id'].unique())

REFERENCE SAMPLE: Function to get end trial difference values for reference sample

In [ ]:
def reference_diff_beg_end(simulation_df, pss_value_list, accuracy_list, plots=True):
    
    diff_df = pd.DataFrame()

    # Initialize figure if plotting is enabled
    #if plots:
    #    fig, axes = plt.subplots(nrows=11, ncols=12, figsize=(22, 18), sharex=False, sharey=False)
    #    fig.suptitle("Staircase 100 vs. 400 (All Available Trials) Across PSS & Accuracy", fontsize=16)


    for i, pss_value in enumerate(pss_value_list):
        df_pss = simulation_df[simulation_df['PSS_value'] == pss_value]

        for j, accuracy in enumerate(accuracy_list):
            df_accuracy = df_pss[df_pss['Accuracy'] == accuracy]

            if df_accuracy.empty:
                continue  # Skip if no data

            # Compute mean 'Current Delay' per staircase and trial
            mean_stim_values = df_accuracy.groupby(['Staircase_name', 'Trial'])['Current Delay'].mean()
            #mean_stim_values = mean_stim_values[mean_stim_values.index.get_level_values('Trial').isin(range(6, 16))]

            # If you want to check the output after filtering
            #print(mean_stim_values)
            # Reshape into a DataFrame for easier handling
            mean_stim_values = mean_stim_values.unstack(level=0)  # Columns = staircase names
            
            # Ensure both staircases (400 & 100) are present
            if '400' in mean_stim_values.columns and '100' in mean_stim_values.columns:
                staircase_400 = mean_stim_values['400'].dropna()
                staircase_100 = mean_stim_values['100'].dropna()
                
            start_400 = mean_stim_values['400'][1]
            end_400 = mean_stim_values['400'][15]
            #print(f"{pss_value} - {accuracy}: 400: {start_400}, {end_400}")

            start_100 = mean_stim_values['100'][1]
            end_100 = mean_stim_values['100'][15]
            #print(f"{pss_value} - {accuracy}: 100: {start_100}, {end_100}")

            start_diff = start_400 - start_100
            end_diff = end_400 - end_100
            
            #print(f"{start_diff}, {end_diff}")
            
            
                       # Create a new row as a DataFrame
            new_row = pd.DataFrame({
                'PSS_Value': [pss_value],
                'Accuracy': [accuracy],
                'start_400': [start_400],
                'end_400': [end_400],
                'start_100': [start_100],
                'end_100': [end_100],
                'start_diff': [start_diff],
                'end_diff': [end_diff]
            })
            
            # Concatenate the new row with the existing DataFrame
            diff_df = pd.concat([diff_df, new_row], ignore_index=True)

            
    return diff_df
   

SIMULATED SAMPLE: Function to compute response counts

In [ ]:
def compute_response_counts_simulated_participants(participants_df, participant_id):
    
    #print(f"Processing participant_id: {participant_id}, num rows: {len(participants_df)}")
    
    # Count the occurrences of each response code per Staircase_name
    response_counts = participants_df.groupby(['Staircase_name', 'Response_code']).size().unstack(fill_value=0)
    
    # Rename columns for 
    response_counts = response_counts.rename(columns={-1: "Before_Response", 
                                                       0: "Same_Response", 
                                                       1: "After_Response"})
    
    # Reset index for merging or visualization
    response_counts = response_counts.reset_index()
    
    # Add participant_id column
    response_counts['participant_id'] = participant_id
    
    return response_counts

SIMUATED SAMPLE: Function to calculate how many participants had >5 or <15 button presses per staircase

In [ ]:
def count_participants_with_values_outside_range_for_staircase_type(pivoted_df, staircase_type='100'):
    """
    This function counts the number of participants whose any of the response columns for the specified staircase type
    (e.g., '100' or '400') fall outside the range [5, 15].

    Parameters:
    pivoted_df : DataFrame
        The pivoted DataFrame with response counts.
    staircase_type : str
        The staircase type ('100' or '400') to filter for.

    Returns:
    int
        The number of participants whose any of the response columns fall outside the range [5, 15].
    """
    # Define the response columns
    response_columns = ['Before_Response', 'Same_Response', 'After_Response']
    
    # Filter the columns based on the staircase type (e.g., '100' or '400')
    staircase_columns = [f"{col}_{staircase_type}" for col in response_columns]
    
    # Check if any of the columns for this staircase type fall outside the range [5, 15]
    condition = (pivoted_df[staircase_columns] <= 5) | (pivoted_df[staircase_columns] >= 15)
    
    # Get the participant IDs where any of the conditions hold true (i.e., any value falls outside [5, 15])
    participants_outside_range = pivoted_df[condition.any(axis=1)].index.tolist()
    
    return participants_outside_range

SIMUATED SAMPLE: Function to calculate how many participants had >5 or <15 button presses in either staircase

In [ ]:
def count_overall_outside_range(pivoted_df):
    """
    This function counts the number of unique participants whose any response column for either staircase type
    ('100' or '400') falls outside the range [5, 15].

    Parameters:
    pivoted_df : DataFrame
        The pivoted DataFrame with response counts.
    
    Returns:
    unique_participants_outside_range : list
        The list of unique participant IDs who meet the condition for either staircase type.
    """
    # Get participants for staircase '100'
    participants_100_outside_range = count_participants_with_values_outside_range_for_staircase_type(pivoted_df, staircase_type='100')

    # Get participants for staircase '400'
    participants_400_outside_range = count_participants_with_values_outside_range_for_staircase_type(pivoted_df, staircase_type='400')

    # Combine the two lists to get unique participants (those who meet either or both conditions)
    unique_participants_outside_range = list(set(participants_100_outside_range) | set(participants_400_outside_range))
    
    return unique_participants_outside_range

SIMULATED SAMPLE: Function to process and plot data for each PSS value, grouped by Staircase_name and Accuracy


In [ ]:
def plot_simulations_with_participant_data(simulation_df, participant_df, participant_id, pss_value_list, accuracy_list, n_iterations, trials_per_iteration=15):
    colors = plt.cm.viridis(np.linspace(0, 1, len(accuracy_list)))  # Color map for accuracy levels
    participant_df = participant_df[participant_df['Participant_ID'] == participant_id]
    
    # Create a grid for the subplots
    n_rows = 3
    n_cols = 3
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(18, 12))
    axs = axs.ravel()  # Flatten the 2D array of axes to 1D for easy iteration

    # Loop through each PSS value
    for i, pss_value in enumerate(pss_value_list):
        ax = axs[i]
        df_pss = simulation_df[simulation_df['PSS_value'] == pss_value]
        
        # Set title for the subplot
        ax.set_title(f'PSS Value: {pss_value} ms', fontsize=14)

        # Loop through each accuracy value
        for accuracy_idx, accuracy in enumerate(accuracy_list):
            # Filter data by accuracy and group by Staircase_name and Trial
            df_accuracy = df_pss[df_pss['Accuracy'] == accuracy]
            mean_stim_values = df_accuracy.groupby(['Staircase_name', 'Trial'])['Current Delay'].mean()
            std_stim_values = df_accuracy.groupby(['Staircase_name', 'Trial'])['Current Delay'].std()

            # Plot each staircase separately
            for staircase_name, group in mean_stim_values.groupby(level=0):
                # Compute 95% confidence intervals
                trial_numbers = group.index.get_level_values(1)
                ci_95 = 1.96 * std_stim_values.loc[staircase_name] / np.sqrt(n_iterations)

                # Plot the mean stimulus values and confidence intervals
                ax.plot(group.index.get_level_values(1), group.values, 
                        label=f'{staircase_name} - Accuracy {accuracy*100:.0f}%', 
                        color=colors[accuracy_idx], 
                        marker='o')

                ax.fill_between(group.index.get_level_values(1), 
                                group.values - ci_95, 
                                group.values + ci_95, 
                                color=colors[accuracy_idx], 
                                alpha=0.2)

        # Overlay participant data (mean of repeated staircases)
        # Group participant data by Modified Staircase_name and Trial
        participant_mean_data = participant_df.groupby(['Modified_Staircase_name', 'Trial'])['Current Delay'].mean()
        participant_std_data = participant_df.groupby(['Modified_Staircase_name', 'Trial'])['Current Delay'].std()

        # Plot each staircase mean with a different line
        for staircase_name in participant_df['Modified_Staircase_name'].unique():
            # Filter participant data for the current staircase
            participant_data_for_staircase = participant_mean_data.loc[staircase_name]
            if len(participant_data_for_staircase) > 1:  # Ensure there is more than 1 data point
                trial_numbers = participant_data_for_staircase.index  # X-axis: Trial numbers
                mean_delays = participant_data_for_staircase.values  # Y-axis: Current Delay values

                # Compute confidence intervals for participant datas
                ci_participant_95 = 1.96 * participant_std_data.loc[staircase_name] / np.sqrt(n_iterations)

                # Plot the participant mean and confidence interval
                ax.plot(trial_numbers, mean_delays, 
                        label=f'Participant ({staircase_name})', 
                        color='red', 
                        linewidth=2, 
                        marker='x')

                ax.fill_between(trial_numbers, 
                                mean_delays - ci_participant_95, 
                                mean_delays + ci_participant_95, 
                                color='red', alpha=0.2)

        # Labeling the subplot
        ax.set_xlabel('Trial')
        ax.set_ylabel('Current Delay (ms)')
        ax.set_ylim(0, 800)  # Set y-axis limit between 0 and 800
        #ax.legend(loc='upper right')

    # Adjust layout and show the plot
    file_name = '/PATH/Trajectory_plots_simulations/PSS_stimulus_progression_sub-' + participant_id + '.png'
    
    plt.tight_layout()
    plt.savefig(file_name)
    plt.show()


SIMULATED SAMPLE: Function to generate 'participant_matrix'. For each participant returns pred_pss_value, pred_accuracy, rmse, rmse_table, and optional heatmap.

In [ ]:
def participant_matrix(df, participant_id, pivot_df, drop = True, plots=False, verbose=False):
    # Filter the data for this participant directly from the DataFrame
    participant_data = df[df['Participant_ID'] == participant_id]
    #print(participant_data)
    if participant_data.empty:
        # If no data is found, return None
        return None, None, None, None, None  # Return None if no data is found

    # Recode the trial numbers: 1-15 for 400ms, 16-30 for 100ms
    participant_data.loc[:, 'Recode_Trial'] = np.where(participant_data['Staircase_name'].str.contains('400'),
                                                       participant_data['Trial'], 
                                                       participant_data['Trial'] + 15)

    # For 400ms (trials 1-15)
    mean_delays_400 = participant_data[participant_data['Staircase_name'].str.contains('400')] \
                        .groupby('Recode_Trial')['Current Delay'].mean()

    # For 100ms (trials 16-30)
    mean_delays_100 = participant_data[participant_data['Staircase_name'].str.contains('100')] \
                        .groupby('Recode_Trial')['Current Delay'].mean()

    # Combine the two mean delay series (400_1, 400_2, and 100_1, 100_2) into one DataFrame
    #mean_delays = mean_delays_400.append(mean_delays_100)
    mean_delays = pd.concat([mean_delays_400, mean_delays_100])

    # Reindex the trials: 1-15 for 400ms, 16-30 for 100ms
    mean_delays.index = np.arange(1, 31)

    if drop:
        #mean_delays = mean_delays.drop([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]) # last 5 trials only
        mean_delays = mean_delays.drop([1, 2, 3, 4, 5, 16, 17, 18, 19, 20]) #trials 5-15

    if verbose: 
        print('Mean Delays:')
        print(mean_delays)
        
    # Create an empty DataFrame to store the RMSE values for each PSS and accuracy level
    rmse_table = pd.DataFrame(index=pivot_df.index.get_level_values('PSS_value').unique(), 
                              columns=[round(i * 0.1, 1) for i in range(11)])

    # Loop through each PSS value and accuracy level, calculate RMSE and store it in the table
    for pss in rmse_table.index:
        for accuracy in rmse_table.columns:
            # Extract the trial indices for the current PSS and accuracy level from pivot_df
            trial_indices = pivot_df.xs(pss, level='PSS_value').loc[:, accuracy].index

            # Extract corresponding mean delay values for those trials
            mean_delay_values = mean_delays.loc[trial_indices].values

            # Extract the corresponding pivot values for the current PSS and accuracy level
            pivot_values = pivot_df.xs(pss, level='PSS_value').loc[:, accuracy].values

            # Calculate RMSE if both arrays are not empty
            if mean_delay_values.size > 0 and pivot_values.size > 0:
                rmse = np.sqrt(np.mean((mean_delay_values - pivot_values) ** 2))
            else:
                rmse = np.nan  # If no data is available, assign NaN

            # Store the RMSE in the table
            rmse_table.loc[pss, accuracy] = rmse
    
    if verbose: 
        display(HTML(rmse_table.to_html()))

    # Ensure the RMSE table is numeric (in case of any issues with non-numeric values)
    rmse_table = rmse_table.apply(pd.to_numeric, errors='coerce')
    
        # 2. Find the lowest 5 RMSE values
    lowest_rmse = rmse_table.unstack().sort_values().head(5)

    # Display the lowest 5 RMSE values along with corresponding PSS and accuracy levels
    lowest_rmse_table = pd.DataFrame(lowest_rmse).reset_index()
    lowest_rmse_table.columns = ['Accuracy_Level', 'PSS_Value', 'RMSE']
    
    if verbose: 
        display(HTML(lowest_rmse_table.to_html()))
        print(f"{lowest_rmse_table[0:1]['PSS_Value']} \n {lowest_rmse_table[0:1]['Accuracy_Level']} \n {lowest_rmse_table[0:1]['RMSE']}")

    # Assuming that pred_pss_value, pred_accuracy, and rmse are the first values from lowest_rmse_table
    pred_pss_value = lowest_rmse_table['PSS_Value'].values[0]
    pred_accuracy = lowest_rmse_table['Accuracy_Level'].values[0]
    rmse = lowest_rmse_table['RMSE'].values[0]
    
    
    # Plot if requested
    if plots:
        rmse_table = rmse_table.iloc[::-1]
        # Create the heatmap
        plt.figure(figsize=(12, 8))  # Adjust figure size if necessary
        sns.heatmap(rmse_table, annot=True, cmap="coolwarm", cbar_kws={'label': 'RMSE'}, fmt='.1f')
        plt.title("Heatmap of RMSE for Different PSS Values and Accuracy Levels")
        plt.xlabel("Accuracy Levels")
        plt.ylabel("PSS Values")
        plt.show()
                
            
        #Create a density plot:
        # Flatten the RMSE table values into a 1D array
        rmse_values = rmse_table.values.flatten().round()
        plt.figure(figsize=(12, 8))
        sns.kdeplot(rmse_values, shade=True, color='skyblue', bw_adjust=0.5)

        # Add annotated vertical lines based on RMSE values from the table
        # Assuming these are the RMSE values where you want the lines
        rmse_table['RMSE'] = pd.to_numeric(lowest_rmse_table['RMSE'], errors='coerce')
        rmse_line_values = lowest_rmse_table['RMSE'].tolist()

        # Plot vertical lines and annotate
        for value in rmse_line_values:
            plt.axvline(x=value, color='red', linestyle='--', linewidth=1)
            plt.text(value + 0.2, 0.002, f'RMSE = {value:.0f}', color='red', rotation=45)

        # Add title and labels
        plt.title("Density Plot of RMSE Values with Annotated Vertical Lines")
        plt.xlabel("RMSE Value")
        plt.ylabel("Density")
        filename = './Simulated_participant_trajectory_plots/' +participant_id + 'trajectory.png'
        plt.savefig(filename)
        # Display the plot
        plt.show()

        
        # Create a scatter plot
        rmse_table_reset = rmse_table.reset_index()

        # Melt the DataFrame so that each row corresponds to an (accuracy, RMSE) pair with a PSS value
        rmse_table_melted = rmse_table_reset.melt(id_vars=['PSS_value'], var_name='Accuracy_Level', value_name='RMSE')

        # Create the scatter plot
        plt.figure(figsize=(10, 6))

        # Use seaborn's scatterplot with color mapped to PSS_value
        sns.scatterplot(data=rmse_table_melted, x='Accuracy_Level', y='RMSE', hue='PSS_value', palette='viridis', s=100, marker='o')

        # Add labels and title
        plt.title("Accuracy vs RMSE (Colored by PSS Value)", fontsize=16)
        plt.xlabel("Accuracy Level", fontsize=14)
        plt.ylabel("RMSE", fontsize=14)

        # Show the plot
        plt.legend(title='PSS Value', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.show()

    return pred_pss_value, pred_accuracy, rmse, rmse_table 




SIMULATED SAMPLE: Function to get p-value of rmse for allocated PSS, relative to the null, for each participant

In [ ]:
def compute_p_values(participant_data, df_vector_across_slices, plotting=False):
    p_values = []
    
    for idx, participant in participant_data.iterrows():
        participant_rmse = participant['RMSE']
        participant_pss = participant['Pred_PSS_Value']
        participant_accuracy = participant['Pred_Accuracy']
        
        # Extract the corresponding distribution from df_vector_across_slices
        try:
            null_vector = df_vector_across_slices.loc[participant_pss, participant_accuracy]
            null_vector = [x for x in null_vector if not np.isnan(x)]  # Remove NaNs if any
        except KeyError:
            print(f"Warning: PSS={participant_pss}, Accuracy={participant_accuracy} not found in df_vector_across_slices.")
            p_values.append(np.nan)
            continue
        
        if len(null_vector) < 30:
            #print(f"Insufficient data for PSS={participant_pss}, Accuracy={participant_accuracy}.")
            # Assign p-value as 1/10,000 if there's no data (extremely unlikely)
            print(f"Extreme case: PSS={participant_pss}, Accuracy={participant_accuracy} Participant RMSE of {participant_rmse:.2f} is out of range of the null distribution.")
            p_values.append(1/10000)  # Reflect extreme unlikeliness
            continue
        
        # Calculate mean and std deviation of the null distribution
        mean_null = np.mean(null_vector)
        std_null = np.std(null_vector, ddof=1)  # Sample std deviation
        
        # Calculate one-tailed p-value (Pr(X < participant_rmse))
        p_value = norm.cdf(participant_rmse, loc=mean_null, scale=std_null)
        
        p_values.append(p_value)
        
        # Optional plotting
        if plotting:
            plt.figure(figsize=(8, 6))
            sns.histplot(null_vector, bins=50, kde=True, color='#F27B5A', label='Null Distribution', alpha=0.1)
            plt.axvline(participant_rmse, color='red', linestyle='--', linewidth=2, label=f'Participant RMSE ({participant_rmse:.2f})')
            plt.title(f'Participant RMSE vs Null Distribution\nPSS={participant_pss}, Accuracy={participant_accuracy}, p-value={p_value:.3f}')
            plt.xlabel('RMSE')
            plt.ylabel('Frequency')
            plt.legend()
            plt.close()  # Close the plot to suppress showing
    
    # Add p-values to the participant_data DataFrame
    participant_data['p_value'] = p_values
    return participant_data

SIMULATED SAMPLE: Function to get p-value of rmse for known PSS, relative to the null, for each participant

In [ ]:
def compute_p_values_true(participant_data, df_vector_across_slices, plotting=False):
    p_values = []
    
    for idx, participant in participant_data.iterrows():
        participant_rmse = participant['True_RMSE']
        participant_pss = participant['PSS_value']
        participant_accuracy = participant['Accuracy']
        
        # Extract the corresponding distribution from df_vector_across_slices
        try:
            null_vector = df_vector_across_slices.loc[participant_pss, participant_accuracy]
            null_vector = [x for x in null_vector if not np.isnan(x)]  # Remove NaNs if any
        except KeyError:
            print(f"Warning: PSS={participant_pss}, Accuracy={participant_accuracy} not found in df_vector_across_slices.")
            p_values.append(np.nan)
            continue
        
        if len(null_vector) < 30:
            #print(f"Insufficient data for PSS={participant_pss}, Accuracy={participant_accuracy}.")
            # Assign p-value as 1/10,000 if there's no data (extremely unlikely)
            print(f"Extreme case: PSS={participant_pss}, Accuracy={participant_accuracy} Participant RMSE of {participant_rmse:.2f} is out of range of the null distribution.")
            p_values.append(1/10000)  # Reflect extreme unlikeliness
            continue
        
        # Calculate mean and std deviation of the null distribution
        mean_null = np.mean(null_vector)
        std_null = np.std(null_vector, ddof=1)  # Sample std deviation
        
        # Calculate one-tailed p-value (Pr(X < participant_rmse))
        p_value = norm.cdf(participant_rmse, loc=mean_null, scale=std_null)
        
        p_values.append(p_value)
        
        # Optional plotting
        if plotting:
            plt.figure(figsize=(8, 6))
            sns.histplot(null_vector, bins=50, kde=True, color='#F27B5A', label='Null Distribution', alpha=0.1)
            plt.axvline(participant_rmse, color='red', linestyle='--', linewidth=2, label=f'Participant RMSE ({participant_rmse:.2f})')
            plt.title(f'Participant RMSE vs Null Distribution\nPSS={participant_pss}, Accuracy={participant_accuracy}, p-value={p_value:.3f}')
            plt.xlabel('RMSE')
            plt.ylabel('Frequency')
            plt.legend()
            plt.close()  # Close the plot to suppress showing
    
    # Add p-values to the participant_data DataFrame
    participant_data['p_value'] = p_values
    return participant_data

Define colour palettes:

In [ ]:
#Referenece:
null_colors = ["#B5D5E0", "#4A738F"]
null_cmap = LinearSegmentedColormap.from_list("custom_heatmap", null_colors, N=256)
data = np.random.rand(10, 10)
sns.heatmap(data, cmap=null_cmap)
plt.show()

#Null:
null_colors = ["#B1DFEE", "#5085E1"]
null_cmap = LinearSegmentedColormap.from_list("custom_heatmap", null_colors, N=256)
data = np.random.rand(10, 10)
sns.heatmap(data, cmap=null_cmap)
plt.show()


#Simulated Participants:
sim_participant_colors = ["#EFD0B6", "#e86a36"]
sim_participant_cmap = LinearSegmentedColormap.from_list("custom_heatmap", sim_participant_colors, N=256)
data = np.random.rand(10, 10)
sns.heatmap(data, cmap=sim_participant_cmap)
plt.show()


# Real Participants:
real_participant_colors = ["#EEB0C4", "#e54e40"]
real_participant_cmap = LinearSegmentedColormap.from_list("custom_heatmap", real_participant_colors, N=256)
data = np.random.rand(10, 10)
sns.heatmap(data, cmap=real_participant_cmap)
plt.show()


# Run analysis:

[If all files have been generated click here](#skip-here)

REFERENCE SAMPLE: 

Generate the reference trajectories and get reference matrix (pivot_df)
Optional plotting
Optional dropping of the first 5 trials

For: PSS = 100-400; Accuracy = 0 - 1.0

⚠️ NOTE ⚠️

If you have already run the analysis and you have the following files - skip to below:
10000_null_simulations_100_400_pred_pss_accuracy.csv
10000_null_simulations_100_400_all_mean_rmse_plot.npy


In [ ]:
# Initialize DataFrames to store accumulated results
final_pred_pss_accuracy = pd.DataFrame(columns=['Participant_ID', 'Pred_PSS_Value', 'Pred_Accuracy', 'RMSE', 'PSS_value', 'Accuracy', 'True_RMSE'])

# Define fixed ranges for the structure
pred_pss_values = np.arange(100, 401, 50)  # Pred_PSS_Value: 0 to 400, increments of 50
pred_accuracies = np.arange(0, 1.1, 0.1).round(1)  # Pred_Accuracy: 0.1 to 1.0, increments of 0.1

# Load simulation data
simulation_df = pd.read_csv('/PATH/Reference_simulations.tsv', sep='\t')
simulation_df['Staircase_name'] = simulation_df['Staircase_name'].astype(str)
#Run reference_matrix function to get PSS and Accuracy trajectories
pivot_df = reference_matrix(simulation_df, pred_pss_values,pred_accuracies, drop = True, plots=True)
#Save file
pivot_df.to_csv("Reference_trajectories_100_400.csv", index=True)  

display(HTML(pivot_df.head().to_html()))

# 🎲 IDENTIFY RANDOM VS NON-RANDOM RESPONSES: Reference Sample

In [ ]:
null_response_counts_df = pd.DataFrame()

for i in tqdm(range(0, 100)):  
    participants_file = f'/PATH/SIMULATED_NULL/10000_random_participant_simulations_{i}.tsv'
    all_null_df = pd.read_csv(participants_file, sep='\t')

    # Get participant IDs
    participant_ids = all_null_df['Participant_ID'].unique().tolist()
        
    # Loop over each participant ID
    for participant_id in participant_ids[0:100]:
        participants_df = all_null_df[all_null_df['Participant_ID'] == participant_id]
        participants_df['Trial'] = participants_df['Trial'].astype(int)
        participants_df['Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]

        # Recode trial numbers
        participants_df.loc[:, 'Recode_Trial'] = np.where(
            participants_df['Staircase_name'].str.contains('400'),
            participants_df['Trial'], 
            participants_df['Trial'] + 15
        )

        # Calculate response counts per Staircase_name and Response_code
        response_counts_df = compute_response_counts(participants_df, participant_id, i)
        null_response_counts_df = pd.concat([null_response_counts_df, response_counts_df], ignore_index=True)

print(null_response_counts_df.head())


# Create Staircase Group column
null_response_counts_df["Modified_Staircase_name"] = null_response_counts_df["Staircase_name"].apply(lambda x: "100" if "100" in x else "400")

# Aggregate response counts per participant and staircase group
agg_df = null_response_counts_df.groupby(["participant_id", "Modified_Staircase_name"])[["Before_Response", "Same_Response", "After_Response"]].sum().reset_index()

# Melt the DataFrame for plotting
agg_df_melted = agg_df.melt(id_vars=["participant_id", "Modified_Staircase_name"], 
                            value_vars=["Before_Response", "Same_Response", "After_Response"], 
                            var_name="Response_Type", 
                            value_name="Count")

# Plot histogram
plt.figure(figsize=(8, 6))
sns.histplot(data=agg_df_melted, x="Count", hue="Modified_Staircase_name", bins=100, kde=False)

plt.xlabel("Response Counts")
plt.ylabel("Frequency")
plt.title("Histogram of Response Counts Split by Staircase pair")
plt.grid(True)
plt.show()

agg_df_melted.describe()


def get_participants_outside_range_melted(df, staircase_name):
    """
    Returns a set of participant IDs whose any Count value for the given staircase_name is outside [5, 15]
    """
    filtered = df[df['Modified_Staircase_name'] == staircase_name]
    
    # Find rows where Count is outside the range [5, 15]
    outside_range = filtered[(filtered['Count'] < 5) | (filtered['Count'] > 15)]
    
    # Return unique participant_ids
    return set(outside_range['participant_id'].unique())

# Get participants with out-of-range responses for staircase '100'
participants_100_outside = get_participants_outside_range_melted(agg_df_melted, '100')
print(f"Participants with values outside [5, 15] for staircase '100': {len(participants_100_outside)}")
print(f"Percentage of total participants: {(len(participants_100_outside)/agg_df_melted['participant_id'].nunique())*100:.2f}%")

# Get participants with out-of-range responses for staircase '400'
participants_400_outside = get_participants_outside_range_melted(agg_df_melted, '400')
print(f"Participants with values outside [5, 15] for staircase '400': {len(participants_400_outside)}")
print(f"Percentage of total participants: {(len(participants_400_outside)/agg_df_melted['participant_id'].nunique())*100:.2f}%")

# Unique participants across either staircase type
unique_outside = participants_100_outside.union(participants_400_outside)
print(f"\nUnique participants with out-of-range values in either staircase: {len(unique_outside)}")

# Percentage of total unique participants
total_participants = agg_df_melted['participant_id'].nunique()
perc_outside = (len(unique_outside) / total_participants) * 100
print(f"Percentage of total participants: {perc_outside:.2f}%")


# 🧠 IDENTIFY COGNITIVE STRATEGY: Reference Sample

Check if trajectories are parallel; diverging; converging

In [ ]:
null_diff_df = pd.DataFrame()

for i in tqdm(range(0,100)):  
    participants_file = f'/PATH/SIMULATED_NULL/10000_random_participant_simulations_{i}.tsv'
    all_null_df = pd.read_csv(participants_file, sep='\t')

    # Get participant IDs
    participant_ids = []
    participant_ids = all_null_df['Participant_ID'].unique().tolist()
    #print(participant_ids)
        
    # Loop over each participant ID
    for participant_id in participant_ids[0:100]:
        participants_df = all_null_df[all_null_df['Participant_ID'] == participant_id]
        #print(participants_df)
        participants_df = participants_df.copy()
        participants_df.loc[:, 'Trial'] = participants_df['Trial'].astype(int)
        participants_df.loc[:, 'Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]
        participants_df.loc[:, 'Recode_Trial'] = np.where(
            participants_df['Staircase_name'].str.contains('400'),
            participants_df['Trial'], 
            participants_df['Trial'] + 15
        )

        #print(participants_df)

         # For 400ms (trials 1-15)
        mean_delays_400 = participants_df[participants_df['Staircase_name'].str.contains('400')] \
                            .groupby('Recode_Trial')['Current Delay'].mean()

        # For 100ms (trials 16-30)
        mean_delays_100 = participants_df[participants_df['Staircase_name'].str.contains('100')] \
                            .groupby('Recode_Trial')['Current Delay'].mean()

        # Combine the two mean delay series (400_1, 400_2, and 100_1, 100_2) into one DataFrame
        mean_delays = pd.concat([mean_delays_400, mean_delays_100])

        # Reindex the trials: 1-15 for 400ms, 16-30 for 100ms
        mean_delays.index = np.arange(1, 31)

        start_diff = mean_delays[1] - mean_delays[16]
        end_diff = mean_delays[15] - mean_delays[30]

        new_row = pd.DataFrame([{
        'Participant_ID': participant_id,
        'start_400': mean_delays[1],
        'end_400':  mean_delays[15],
        'start_100':  mean_delays[16],
        'end_100': mean_delays[30],
        'start_diff': start_diff,
        'end_diff': end_diff
        }])

        null_diff_df = pd.concat([null_diff_df, new_row], ignore_index=True)

#Calculate: The distance moved (-ve = converging; +ve = diverging)
null_diff_df['movement'] = null_diff_df['end_diff'] - null_diff_df['start_diff'] 

# Parallel: End_diff = Start_diff = 300ms
null_parallel_movers = null_diff_df[((null_diff_df["end_diff"] == 300) & (null_diff_df["end_400"] != 400))]

# Divergent: End_diff > Start_diff 
null_divergent_movers = null_diff_df[(null_diff_df["end_diff"] >= 301)]

# Convergent: End_diff < Start_diff 
null_converge_movers = null_diff_df[(null_diff_df["end_diff"] < 300)]

null_diff_df.to_csv("Null_diff_df_100_400.csv", index=False)  

#Summary:
print(len(null_parallel_movers),   len(null_parallel_movers)/ len(null_diff_df)*100,'%: Parallel lines')
print(len(null_divergent_movers)/ len(null_diff_df)*100,'%: Have directions away from converging i.e. acc less than 0.3')
print(len(null_converge_movers)/ len(null_diff_df)*100,'%: Have directions towards converging i.e. acc > than 0.3')
print('\n\nMean end delay difference of null: ', null_diff_df['end_diff'].mean())
print('STD end delay difference of null: ', null_diff_df['end_diff'].std())

#Parallel:
print('\n\nMean end delay of 400 staircase of parallel movers: ', null_parallel_movers['end_400'].mean())
print('Mean end delay 100 staircase of parallel movers: ',null_parallel_movers['end_100'].mean())

#Divergent:
print('\n\nMean change from 300ms diff of divergent movers: ',null_divergent_movers['movement'].mean())
print('STD change from 300ms diff of divergent movers: ',null_divergent_movers['movement'].std())
print('Median end delay difference of divergent movers: ', null_divergent_movers['end_diff'].median())
print('Min end delay difference of divergent movers: ',null_converge_movers['end_diff'].min())
print('Max end delay difference of divergent movers: ',null_converge_movers['end_diff'].max())

#Convergent:
print('\n\nMean change from 300ms diff of convergent movers: ', null_converge_movers['movement'].mean())
print('STF change from 300ms diff of convergent movers: ', null_converge_movers['movement'].std())
print('Median end delay difference of convergent movers: ', null_converge_movers['end_diff'].median())
print('Mean end delay difference of convergent movers: ',null_converge_movers['end_diff'].mean())
print('STD end delay difference of convergent movers: ',null_converge_movers['end_diff'].std())
print('Min end delay difference of convergent movers: ',null_converge_movers['end_diff'].min())
print('Max end delay difference of convergent movers: ',null_converge_movers['end_diff'].max())
count = (null_diff_df['end_diff'] <= 0).sum()
print('Number of convergent movers who converge :',count)


# Plot histogram
plt.figure(figsize=(6,4))
sns.histplot(ref_diff_df['end_diff'].to_numpy(), bins=150, kde=False, color="#4A738F")
plt.xlabel("End Difference (ms)", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("End difference in \n Reference Simulations", fontsize=14)
plt.show()

# Scatter plot
plt.figure(figsize=(6,4))
plt.axhspan(299, 301, color='grey', alpha=0.3)
plt.axhspan(301, 600, color='lightgrey', alpha=0.3)
scatter = sns.scatterplot(
    x=ref_diff_df['PSS_Value'],  # Using index as x-axis
    y=ref_diff_df["end_diff"],
    hue=ref_diff_df["Accuracy"],
    palette="viridis",
    sizes=(20, 200),  # Adjust size range
    edgecolor="black"
)
plt.xlabel("PSS")
plt.ylabel("End Difference (ms)")
plt.title("End difference in \n Reference Simulations")
plt.legend(title="Accuracy", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()

# ❤️ASSIGN PSS AND ACCURACY VALUES: Reference Sample

In [ ]:
all_mean_dfs = []

for i in tqdm(range(0,100)):  
    participants_file = f'/PATH/SIMULATED_NULL/10000_random_participant_simulations_{i}.tsv'
    participants_df = pd.read_csv(participants_file, sep='\t')
    participants_df['Trial'] = participants_df['Trial'].astype(int)
    participants_df['Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]
    
    # Recode trial numbers
    participants_df.loc[:, 'Recode_Trial'] = np.where(
        participants_df['Staircase_name'].str.contains('400'),
        participants_df['Trial'], 
        participants_df['Trial'] + 15
    )
    
    # Get participant IDs
    participant_ids = participants_df['Participant_ID'].unique().tolist()
    
    # Temporary storage for current iteration's results
    Pred_PSS_Accuracy = pd.DataFrame(columns=['Participant_ID', 'Pred_PSS_Value', 'Pred_Accuracy', 'RMSE'])
    rmse_all_table = pd.DataFrame()
    
    # Loop over each participant ID
    for participant_id in participant_ids[0:100]:
        # Call your participant_matrix function (assume it's already defined elsewhere)
        pred_pss_value, pred_accuracy, rmse, rmse_table = participant_matrix(
            participants_df, participant_id, pivot_df, drop=True, plots=False, verbose=False
        )
        
        # Append results to Pred_PSS_Accuracy
        Pred_PSS_Accuracy = Pred_PSS_Accuracy.append({
            'Participant_ID': participant_id,
            'Pred_PSS_Value': pred_pss_value,
            'Pred_Accuracy': pred_accuracy,
            'RMSE': rmse
        }, ignore_index=True)
        
        # Append results to rmse_all_table
        #rmse_table['Participant_ID'] = participant_id
        #rmse_all_table = pd.concat([rmse_all_table, rmse_table], ignore_index=False)  
    
    # Create group structure for aligning means
    group_structure = pd.DataFrame(
        [(v, a) for v in pred_pss_values for a in pred_accuracies],
        columns=['Pred_PSS_Value', 'Pred_Accuracy']
    )
    
    # Group by and calculate means
    grouped_rmse_mean = Pred_PSS_Accuracy.groupby(["Pred_PSS_Value", "Pred_Accuracy"])["RMSE"].mean().reset_index()
    file_mean = group_structure.merge(grouped_rmse_mean, on=['Pred_PSS_Value', 'Pred_Accuracy'], how='left')

    # Pivot to create mean_df
    mean_df = file_mean.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="RMSE")

    # Convert mean_df to a slice of the 3D matrix
    slice_matrix = np.full((len(pred_pss_values), len(pred_accuracies)), np.nan)
    for pss_val, row in mean_df.iterrows():
        for acc_val, rmse in row.items():
            if not np.isnan(rmse):
                slice_matrix[pss_index[pss_val], accuracy_index[acc_val]] = rmse

    # Add the slice to the list
    all_mean_dfs.append(slice_matrix)
    # Add list of PSS_Accuracy to mega list
    final_pred_pss_accuracy = pd.concat([final_pred_pss_accuracy, Pred_PSS_Accuracy], ignore_index=True)
    
    
    final_pred_pss_accuracy.to_csv("10000_null_simulations_100_400_pred_pss_accuracy.csv", index=False)  

np.save('/PATH/10000_null_simulations_100_400_all_mean_rmse_plot.npy', all_mean_dfs)

print('Done!')

In [ ]:
# Create frequency counts for Predicted Accuracy vs. Predicted PSS Value
grouped_pred = null_pred_pss_accuracy.groupby(["Pred_PSS_Value", "Pred_Accuracy"]).size().reset_index(name="Frequency")

# Pivot the table for heatmap structure
heatmap_data_pred = grouped_pred.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="Frequency").fillna(0)
heatmap_data_pred = heatmap_data_pred.iloc[::-1]

fig_width_cm = 8
fig_height_cm = 6
fig_size_in = (fig_width_cm / 2.54, fig_height_cm / 2.54)

# Define full range of accuracy values: 0 to 100 in steps of 10
full_accuracy_vals = np.arange(0, 101, 10)
full_accuracy_props = full_accuracy_vals / 100

# Ensure all expected columns are represented in the heatmap
heatmap_data_pred = heatmap_data_pred.reindex(columns=full_accuracy_props, fill_value=0)

# Plot heatmap for Predicted Accuracy vs. Predicted PSS Value
plt.figure(figsize=fig_size_in, dpi=300)
ax = sns.heatmap(
    heatmap_data_pred, 
    cmap=null_cmap, 
    annot=True, 
    fmt=".0f", 
    vmin=0, 
    vmax=1000, 
    annot_kws={"size": 5},
    cbar_kws={"label": "Frequency of sample"}
)

ax.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax.set_xticklabels(full_accuracy_vals, fontsize=7)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=7)
ax.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax.collections[0].colorbar.ax.tick_params(labelsize=7)

plt.tight_layout()
plt.savefig("10000_null_simulations_100_400_pred_pss_accuracy.png")
plt.show()

# --- Mean RMSE heatmap ---
null_mean_across_slices = np.nanmean(all_mean_dfs, axis=0)
null_mean_df_result = pd.DataFrame(
    null_mean_across_slices,
    index=pred_pss_values,
    columns=pred_accuracies
)
null_mean_df_result = null_mean_df_result.reindex(columns=full_accuracy_props, fill_value=0).iloc[::-1]

plt.figure(figsize=fig_size_in, dpi=300)
ax2 = sns.heatmap(
    null_mean_df_result, 
    annot=True, 
    fmt=".1f", 
    cmap=null_cmap,  
    annot_kws={"size": 5},
    cbar_kws={'label': 'Mean RMSE'}, 
    vmin=0
)

ax2.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax2.set_xticklabels(full_accuracy_vals, fontsize=7)
ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=7)
ax2.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax2.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax2.collections[0].colorbar.ax.tick_params(labelsize=7)

plt.tight_layout()
plt.savefig("10000_null_simulations_mean_RMSE_across_slices.png")
plt.show()


In [ ]:
## PLOTS FOR THE NULL DISTRIBUTION 
# Initialize an empty list to store the result
vector_across_slices = []

# Loop over each (y, x) position (same logic as in the original code)
for y in range(null_mean_df_result.shape[1]):  # 12 rows
    row = []
    for x in range(null_mean_df_result.shape[2]):  # 11 columns
        values_at_pos = null_mean_df_result[:, y, x].tolist()
        row.append(values_at_pos)
    vector_across_slices.append(row)

df_vector_across_slices = pd.DataFrame(vector_across_slices)
df_vector_across_slices.index = pred_pss_values  # Assuming `pred_pss_values` is the list of index labels
df_vector_across_slices.columns = pred_accuracies  # Assuming `pred_accuracies` is the list of column labels

# Set the number of rows and columns for the grid of subplots
num_rows = df_vector_across_slices.shape[0]
num_cols = df_vector_across_slices.shape[1]

# Calculate the min/max values for the x and y axis ranges
min_xvalue = 0
max_xvalue = math.ceil(np.nanmax([item for sublist in df_vector_across_slices.values for item in sublist]) + 0.05)
max_yvalue = 0

# Find max frequency for scaling y-axis
for cell_row in range(num_rows):
    for cell_col in range(num_cols):
        cell_values = df_vector_across_slices.iloc[cell_row, cell_col]
        hist, bin_edges = np.histogram(cell_values, bins=50, range=(min_xvalue, max_xvalue))
        max_yvalue = max(max_yvalue, hist.max())

# Create figure and axes for subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15,10))

# Loop through each subplot to create histograms
for cell_row in range(num_rows):
    for cell_col in range(num_cols):
        cell_values = df_vector_across_slices.iloc[cell_row, cell_col]
        ax = axes[num_rows - 1 - cell_row, cell_col]  # Flip the row order

        # Plot the histogram for the null distribution
        sns.histplot(cell_values, bins=50, kde=True, color='#5085E1', edgecolor='#5085E1', alpha=0.7, ax=ax, 
                     binrange=(min_xvalue, max_xvalue))
        
        # Remove axis labels and ticks for a clean look
        ax.set_xticks([]) 
        ax.set_yticks([]) 
        ax.set_ylabel('')

        # Set axis limits
        ax.set_xlim(min_xvalue, max_xvalue)
        ax.set_ylim(0, max_yvalue)

        # Clean grid appearance
        if cell_col == num_cols - 1:
            ax.spines['top'].set_visible(True)
            ax.spines['right'].set_visible(True)
            ax.spines['left'].set_visible(True)
            ax.spines['bottom'].set_visible(True)
        else:
            ax.spines['top'].set_visible(True)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(True)
            ax.spines['bottom'].set_visible(True)

        if cell_row == 0 and cell_col == 0:
            ax.set_xticks(np.linspace(min_xvalue, max_xvalue, num=2))
        else:
            ax.set_xticks([])

        if cell_col == 0 and cell_row == 0:
            ax.set_yticks(np.linspace(0, max_yvalue, num=2))

# Left Y-axis (Pred_PSS_Value)
ax_left = fig.add_axes([0.065, 0.1, 0.01, 0.75])
ax_left.set_xticks([])  
ax_left.tick_params(axis="y", direction="out", length=12, width=1.2)
ax_left.set_yticks(np.arange(num_rows) + 0.5)
ax_left.set_yticklabels(pred_pss_values, fontsize=10, ha='right')
ax_left.set_ylabel("Predicted PSS Value\nNumber of instances", fontsize=12, labelpad=20)
ax_left.tick_params(left=False, labelleft=True, right=False, labelright=False)
ax_left.spines['top'].set_visible(False)
ax_left.spines['bottom'].set_visible(False)
ax_left.spines['left'].set_visible(False)
ax_left.spines['right'].set_visible(False)

# Bottom X-axis (Pred_Accuracy)
ax_bottom = fig.add_axes([0.16, 0.06, 0.7, 0.01])
ax_bottom.set_yticks([])  
ax_bottom.set_xticks(np.arange(num_cols))
ax_bottom.set_xticklabels(pred_accuracies, rotation=0, fontsize=10, ha='center')
ax_bottom.set_xlabel("RMSE\nPredicted Accuracy", fontsize=12, labelpad=15)
ax_bottom.tick_params(axis="x", direction="inout", length=6, width=1.2)
ax_bottom.spines['top'].set_visible(False)
ax_bottom.spines['bottom'].set_visible(False)
ax_bottom.spines['left'].set_visible(False)
ax_bottom.spines['right'].set_visible(False)

# Adjust layout to fit the grid tightly
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("Null_distribution_plots.png")
plt.show()

# 🎲 IDENTIFY RANDOM VS NON-RANDOM RESPONSES: Simulated Sample

In [ ]:
sim_participant_response_counts_df = pd.DataFrame()

sim_participants_file = f'/PATH/10000_simulated_participants.tsv'
sim_participants_df = pd.read_csv(sim_participants_file, sep='\t')

sim_participants_df['Trial'] = sim_participants_df['Trial'].astype(int)
sim_participants_df['Modified_Staircase_name'] = sim_participants_df['Staircase_name'].str[:-2]

# Recode trial numbers
sim_participants_df.loc[:, 'Recode_Trial'] = np.where(
    sim_participants_df['Staircase_name'].str.contains('400'),
    sim_participants_df['Trial'], 
    sim_participants_df['Trial'] + 15
)
# Get participant IDs
participant_ids = sim_participants_df['Participant_ID'].unique().tolist()
#print(participant_ids)

# Loop over each participant ID
for participant_id in tqdm(range(1,10001)):
    ID_string = 'Participant_' + str(participant_id)
    participants_df = sim_participants_df[sim_participants_df['Participant_ID'] == ID_string]
    #print(participants_df)

    participants_df['Trial'] = participants_df['Trial'].astype(int)
    participants_df['Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]
    
    # Recode trial numbers
    participants_df.loc[:, 'Recode_Trial'] = np.where(
        participants_df['Staircase_name'].str.contains('400'),
        participants_df['Trial'], 
        participants_df['Trial'] + 15
    )
    
    # Get participant IDs
    #participant_data = participants_df[participants_df['Participant_ID'] == participant_id]

    # Calculate response counts per Staircase_name and Response_code
    response_counts_df = compute_response_counts_simulated_participants(participants_df, participant_id)
    sim_participant_response_counts_df = pd.concat([sim_participant_response_counts_df, response_counts_df], ignore_index=True)

print(sim_participant_response_counts_df.head())
sim_participant_response_counts_df.to_csv('sim_participant_response_counts_df.csv')

In [ ]:

# Create the 'Modified_Staircase_name' column for null_response_counts_df
null_response_counts_df['Modified_Staircase_name'] = null_response_counts_df['Staircase_name'].str[:-2]

# Separate the data into two groups based on the 'Modified_Staircase_name' prefix
df_100_null = null_response_counts_df[null_response_counts_df['Modified_Staircase_name'].str.startswith('100')]
df_400_null = null_response_counts_df[null_response_counts_df['Modified_Staircase_name'].str.startswith('400')]

# Group by participant_id and Modified_Staircase_name, and sum the responses
summed_100_null = df_100_null.groupby(['participant_id', 'Modified_Staircase_name'])[['Before_Response', 'Same_Response', 'After_Response']].sum()
summed_400_null = df_400_null.groupby(['participant_id', 'Modified_Staircase_name'])[['Before_Response', 'Same_Response', 'After_Response']].sum()

# Now we concatenate the results for both 100 and 400 staircases
combined_sums_null = pd.concat([summed_100_null, summed_400_null], axis=0)
combined_sums_null.reset_index(inplace=True)

# Pivot the table so that each participant has one row, and each staircase is a separate column
pivoted_null_df = combined_sums_null.pivot_table(index='participant_id', 
                                                 columns='Modified_Staircase_name', 
                                                 values=['Before_Response', 'Same_Response', 'After_Response'], 
                                                 aggfunc='sum')


pivoted_null_df.columns = ['_'.join(col).strip() for col in pivoted_null_df.columns.values]
#print(pivoted_null_df.head())

mean_null_row = pivoted_null_df.mean().to_frame().T
mean_null_row.index = ['Null']  # Rename the index to 'Null'

# Now merge the mean row with the pivoted_df (participant's data)
pivoted_df_with_null = pd.concat([pivoted_df, mean_null_row])
# Show the resulting DataFrame
#print(pivoted_df_with_null)
 
 
 
sim_participant_response_counts_df = pd.read_csv('sim_participant_response_counts_df.csv')
sim_participant_response_counts_df['Modified_Staircase_name'] = sim_participant_response_counts_df['Staircase_name'].str[:-2]
#print(sim_participant_response_counts_df)

# Separate the data into two groups based on the 'Modified_Staircase_name' prefix
df_100 = sim_participant_response_counts_df[sim_participant_response_counts_df['Modified_Staircase_name'].str.startswith('100')]
df_400 = sim_participant_response_counts_df[sim_participant_response_counts_df['Modified_Staircase_name'].str.startswith('400')]
#print(df_100)

# Group by participant_id and Modified_Staircase_name, and sum the responses
summed_100 = df_100.groupby(['participant_id', 'Modified_Staircase_name'])[['Before_Response', 'Same_Response', 'After_Response']].sum()
summed_400 = df_400.groupby(['participant_id', 'Modified_Staircase_name'])[['Before_Response', 'Same_Response', 'After_Response']].sum()

# Now we concatenate the results for both 100 and 400 staircases
combined_sums = pd.concat([summed_100, summed_400], axis=0)

# Reset index to make 'participant_id' and 'Modified_Staircase_name' regular columns
combined_sums.reset_index(inplace=True)

# Pivot the table so that each participant has one row, and each staircase is a separate column
pivoted_df = combined_sums.pivot_table(index='participant_id', 
                                       columns='Modified_Staircase_name', 
                                       values=['Before_Response', 'Same_Response', 'After_Response'], 
                                       aggfunc='sum')

# Flatten the multi-index columns
pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
#print(pivoted_df)


# Get participants whose any response values for staircase '100' fall outside the range [5, 15]
participants_100_outside_range = count_participants_with_values_outside_range_for_staircase_type(pivoted_df_with_null, staircase_type='100')
print(f"Participants with any response values outside the range [5, 15] for staircase '100': {len(participants_100_outside_range)}")

# Get participants whose any response values for staircase '400' fall outside the range [5, 15]
participants_400_outside_range = count_participants_with_values_outside_range_for_staircase_type(pivoted_df_with_null, staircase_type='400')
print(f"Participants with any response values == or outside the range [5, 15] for staircase '400': {len(participants_400_outside_range)}")

# Get all unique participants whose any response values fall outside the range [5, 15] for either staircase type
overall_outside_range_participants = count_overall_outside_range(pivoted_df_with_null)
print(f"\nOverall unique participants with any response values == or outside the range [5, 15] (for both '100' or '400'): {len(overall_outside_range_participants)}")

print("\nNumber of unique participants who meet the condition (either or both staircase types):")
perc = (len(overall_outside_range_participants)/ len(pivoted_df)) * 100
print(perc, '%')

# Optionally print the actual IDs of participants
#print("Unique participants who meet the condition (either or both staircase types):")
#print(overall_outside_range_participants)


#participant_response_counts_df
all_ids = set(sim_participant_response_counts_df['participant_id'].astype(str))
keep_ids = set(overall_outside_range_participants)

all_ids = set(sim_participant_response_counts_df['participant_id'].astype(str))
keep_ids = set(str(k) for k in overall_outside_range_participants)

# Get those NOT in overall_outside_range_participants
exclude_ids_random = list(all_ids - keep_ids)
#print('Exclude: ', exclude_ids_random)
print('Exclude: ', len(exclude_ids_random), (len(exclude_ids_random))/10000*100)

In [ ]:
null_colors = ['#87afe6', '#b1d8ec', '#6991e1']
sim_colors = ['#e49971', '#ecd1b7', '#e17a4b']

def get_cols_by_response_and_staircase(df, response_type, staircase_prefix):
    # Return list of columns matching exact pattern ResponseType_StaircasePrefix
    return [col for col in df.columns if col == f"{response_type}_{staircase_prefix}"]

# Plot for 100 staircases
plt.figure(figsize=(12, 6))
for i, response in enumerate(['Before_Response', 'Same_Response', 'After_Response']):
    cols = get_cols_by_response_and_staircase(pivoted_null_df, response, '100')
    if len(cols) == 0:
        print(f"No columns found for {response} 100")
        continue
    # cols is a list with one element (the exact column name)
    sns.kdeplot(pivoted_null_df[cols[0]], fill=True, alpha=0.3, color=null_colors[i], label=f'Null 100 - {response.split("_")[0]}')
    sns.kdeplot(pivoted_df[cols[0]], fill=True, alpha=0.3, color=sim_colors[i], label=f'Sim 100 - {response.split("_")[0]}')
plt.title("KDE of '100' Staircases Responses")
plt.xlabel('Response Value')
plt.ylabel('Density')
plt.xlim(0, 30)
plt.legend()
sns.despine()
plt.tight_layout()
plt.show()

# Plot for 400 staircases
plt.figure(figsize=(12, 6))
for i, response in enumerate(['Before_Response', 'Same_Response', 'After_Response']):
    cols = get_cols_by_response_and_staircase(pivoted_null_df, response, '400')
    if len(cols) == 0:
        print(f"No columns found for {response} 400")
        continue
    # cols is a list with one element (the exact column name)
    sns.kdeplot(pivoted_null_df[cols[0]], fill=True, alpha=0.3, color=null_colors[i], label=f'Null 400 - {response.split("_")[0]}')
    sns.kdeplot(pivoted_df[cols[0]], fill=True, alpha=0.3, color=sim_colors[i], label=f'Sim 400 - {response.split("_")[0]}')
plt.title("KDE of '400' Staircases Responses")
plt.xlabel('Response Value')
plt.ylabel('Density')
plt.xlim(0, 30)
plt.legend()
sns.despine()
plt.tight_layout()
plt.show()


In [ ]:
# Set colors
null_colors = ['#87afe6', '#b1d8ec', '#6991e1']
sim_colors = ['#e49971', '#ecd1b7', '#e17a4b']

# Initialize the figure
plt.figure(figsize=(12, 6))

# Plot null group responses
sns.kdeplot(pivoted_null_df.filter(like='Before_Response').mean(axis=1), 
            fill=True, color=null_colors[0], alpha=0.3, label='Null - Before')
sns.kdeplot(pivoted_null_df.filter(like='Same_Response').mean(axis=1), 
            fill=True, color=null_colors[1], alpha=0.3, label='Null - Same')
sns.kdeplot(pivoted_null_df.filter(like='After_Response').mean(axis=1), 
            fill=True, color=null_colors[2], alpha=0.3, label='Null - After')

# Plot simulated participants group responses
sns.kdeplot(pivoted_df.filter(like='Before_Response').mean(axis=1), 
            fill=True, color=sim_colors[0], alpha=0.3, label='Simulated - Before')
sns.kdeplot(pivoted_df.filter(like='Same_Response').mean(axis=1), 
            fill=True, color=sim_colors[1], alpha=0.3, label='Simulated - Same')
sns.kdeplot(pivoted_df.filter(like='After_Response').mean(axis=1), 
            fill=True, color=sim_colors[2], alpha=0.3, label='Simulated - After')

# Plot settings
plt.title('KDE of Response Types: Null vs Simulated Participants')
plt.xlabel('Sum of Responses')
plt.ylabel('Density')
plt.xlim(0, 30)
plt.legend()
sns.despine()  

plt.tight_layout()
plt.show()


# 🧠 IDENTIFY COGNITIVE STRATEGY: Simulated Sample

Check if trajectories are parallel; diverging; converging

In [ ]:
sim_participant_diff_df = pd.DataFrame()

sim_participants_file = f'/PATH/10000_simulated_participants.tsv'
sim_participants_df = pd.read_csv(sim_participants_file, sep='\t')
print(sim_participants_df)
sim_participants_df['Trial'] = sim_participants_df['Trial'].astype(int)
sim_participants_df['Modified_Staircase_name'] = sim_participants_df['Staircase_name'].str[:-2]

# Recode trial numbers
sim_participants_df.loc[:, 'Recode_Trial'] = np.where(
    sim_participants_df['Staircase_name'].str.contains('400'),
    sim_participants_df['Trial'], 
    sim_participants_df['Trial'] + 15
)

# Get participant IDs
participant_ids = sim_participants_df['Participant_ID'].unique().tolist()

# Loop over each participant ID
for participant_id in participant_ids[0:10001]:

    participants_df = sim_participants_df[sim_participants_df['Participant_ID'] == participant_id]
    participants_df['Trial'] = participants_df['Trial'].astype(int)
    participants_df['Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]
    
    # Recode trial numbers
    participants_df.loc[:, 'Recode_Trial'] = np.where(
        participants_df['Staircase_name'].str.contains('400'),
        participants_df['Trial'], 
        participants_df['Trial'] + 15
    )
    
    # Get participant IDs
    participant_ids = participants_df['Participant_ID'].unique().tolist()
    

     # For 400ms (trials 1-15)
    mean_delays_400 = participants_df[participants_df['Staircase_name'].str.contains('400')] \
                        .groupby('Recode_Trial')['Current Delay'].mean()

    # For 100ms (trials 16-30)
    mean_delays_100 = participants_df[participants_df['Staircase_name'].str.contains('100')] \
                        .groupby('Recode_Trial')['Current Delay'].mean()

    # Combine the two mean delay series (400_1, 400_2, and 100_1, 100_2) into one DataFrame
    mean_delays = pd.concat([mean_delays_400, mean_delays_100], ignore_index=True)

    # Reindex the trials: 1-15 for 400ms, 16-30 for 100ms
    mean_delays.index = np.arange(1, 31)

    start_diff = mean_delays[1] - mean_delays[16]
    end_diff = mean_delays[15] - mean_delays[30]
    # Create a new DataFrame row
    new_row = pd.DataFrame({
        'Participant_ID': [participant_id],
        'start_400': [mean_delays[1]],
        'end_400': [mean_delays[15]],
        'start_100': [mean_delays[16]],
        'end_100': [mean_delays[30]],
        'start_diff': [start_diff],
        'end_diff': [end_diff]
    })
    
    # Concatenate the new row with the existing DataFrame
    sim_participant_diff_df = pd.concat([sim_participant_diff_df, new_row], ignore_index=True)

#⚠️ IF EXCLUDING PARTICIPANTS WHO HAVE RANDOM BUTTON PRESS: RUN THIS LINE
# Convert keep_ids (numbers or strings) to formatted Participant_ID strings
keep_ids_str = {f"Participant_{i}" for i in keep_ids}

# Filter DataFrame by these formatted IDs
sim_participant_diff_df = sim_participant_diff_df[
    sim_participant_diff_df['Participant_ID'].isin(keep_ids_str)
]


#Calculate: The distance moved (-ve = converging; +ve = diverging)
sim_participant_diff_df['movement'] = sim_participant_diff_df['end_diff'] - sim_participant_diff_df['start_diff'] 

# Parallel: End_diff = Start_diff = 300ms
sim_participant_parallel_movers = sim_participant_diff_df[((sim_participant_diff_df["end_diff"] == 300) & (sim_participant_diff_df["end_400"] != 400))]
print(sim_participant_parallel_movers)
exclude_ids_parallel = set(sim_participant_parallel_movers['Participant_ID'].astype(str))

# Divergent: End_diff > Start_diff 
sim_participant_divergent_movers = sim_participant_diff_df[(sim_participant_diff_df["end_diff"] >= 301)]

# Convergent: End_diff < Start_diff 
sim_participant_converge_movers = sim_participant_diff_df[(sim_participant_diff_df["end_diff"] < 300)]

sim_participant_diff_df.to_csv("10000_simulated_participants_diff_df_100_400.csv", index=False)  


In [ ]:
print(len(sim_participant_diff_df), 'Simulated participants included')

#Summary:
print((len(sim_participant_parallel_movers)),':', len(sim_participant_parallel_movers)/ 10000*100,'%: Parallel lines')
print((len(sim_participant_divergent_movers)),':',len(sim_participant_divergent_movers)/ 10000*100,'%: Have directions away from converging i.e. acc less than 0.3')
print((len(sim_participant_converge_movers)),':',len(sim_participant_converge_movers)/ 10000*100,'%: Have directions towards converging i.e. acc > than 0.3')
print('\n\nMean end delay difference of sim participants: ', sim_participant_diff_df['end_diff'].mean())
print('STD end delay difference of sim participants: ', sim_participant_diff_df['end_diff'].std())

#Parallel:
print('\n\nMean end delay of 400 staircase of parallel movers: ', sim_participant_parallel_movers['end_400'].mean())
print('Mean end delay 100 staircase of parallel movers: ',sim_participant_parallel_movers['end_100'].mean())

#Divergent:
print('\n\nMean change from 300ms diff of divergent movers: ',sim_participant_divergent_movers['movement'].mean())
print('STD change from 300ms diff of divergent movers: ',sim_participant_divergent_movers['movement'].std())
print('Median end delay difference of divergent movers: ', sim_participant_divergent_movers['end_diff'].median())
print('Min end delay difference of divergent movers: ',sim_participant_divergent_movers['end_diff'].min())
print('Max end delay difference of divergent movers: ',sim_participant_divergent_movers['end_diff'].max())

#Convergent:
print('\n\nMean change from 300ms diff of convergent movers: ', sim_participant_converge_movers['movement'].mean())
print('STF change from 300ms diff of convergent movers: ', sim_participant_converge_movers['movement'].std())
print('Median end delay difference of convergent movers: ', sim_participant_converge_movers['end_diff'].median())
print('Mean end delay difference of convergent movers: ',sim_participant_converge_movers['end_diff'].mean())
print('STD end delay difference of convergent movers: ',sim_participant_converge_movers['end_diff'].std())
print('Min end delay difference of convergent movers: ',sim_participant_converge_movers['end_diff'].min())
print('Max end delay difference of convergent movers: ',sim_participant_converge_movers['end_diff'].max())
count = (sim_participant_diff_df['end_diff'] <= 0).sum()
print('Number of convergent movers who converge completely :',count)


# Plot histogram
plt.figure(figsize=(6, 4))
sns.histplot(ref_diff_df['end_diff'], bins=150, kde=False, color="#478590")
sns.histplot(sim_participant_diff_df['end_diff'], bins=150, kde=False, color="#e86a36")
plt.xlabel("End Difference (ms)", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("End difference in \n Reference and Simulated Participant Simulations", fontsize=14)
plt.show()

# Scatter plot
plt.figure(figsize=(6, 4))
plt.axhspan(299, 301, color='grey', alpha=0.3)
plt.axhspan(301, 600, color='lightgrey', alpha=0.3)
scatter = sns.scatterplot(
    x=ref_diff_df['PSS_Value'],  # Using index as x-axis
    y=ref_diff_df["end_diff"],
    hue=ref_diff_df["Accuracy"],
    palette="viridis",
    sizes=(20, 200),  # Adjust size range
    edgecolor="black"
)
scatter = sns.scatterplot(
    x=sim_participant_diff_df['Participant_ID'],  # Using index as x-axis
    y=sim_participant_diff_df["end_diff"],
    #hue=null_diff_df["Accuracy"],
    color="#e86a36",
    sizes=(20, 200),  # Adjust size range
    edgecolor="black", 
    alpha = 0.1
)
plt.xlabel("PSS")
plt.ylabel("End Difference (ms)")
plt.title("End difference in \n Simulated Participant Simulations")
plt.legend(title="Accuracy", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()


# Density Plot: Divergent vs Convergent
plt.figure(figsize=(6, 4))
sns.kdeplot(null_diff_df['end_diff'], color="#5085E1")#, common_norm = True)
sns.kdeplot(sim_participant_divergent_movers['end_diff'], color="#e86a36", linestyle="--", kde= False)
sns.kdeplot(sim_participant_converge_movers['end_diff'], color="#e86a36", linestyle="-",kde= False)
plt.ylim(0, 0.014)

# Labels and title
plt.xlabel("Difference between mean 400 and 100 staircases at trial 15 (ms)", fontsize=12)
plt.ylabel("Density", fontsize=12)
#plt.title("Difference between mean 400 and 100 staircases at trial 15 \n Divergent vs Convergent Simulated Participants", fontsize=14)


# ❤️ASSIGN PSS AND ACCURACY VALUES: Simulated Sample

In [ ]:
# Define fixed ranges for the structure
pred_pss_values = np.arange(100, 401, 50)  # Pred_PSS_Value: 0 to 400, increments of 50
pred_accuracies = np.arange(0, 1.1, 0.1).round(1)  # Pred_Accuracy: 0.0 to 1.0, increments of 0.1


correct_columns = [
    "Participant_ID", "PSS_value", "Accuracy", "Staircase_name", "Trial", 
    "Current Delay", "Button", "Response_code", "Modified_Staircase_name", "Recode_Trial"
]

sim_participants_file = f'/PATH/10000_simulated_participants.tsv'
sim_participants_df = pd.read_csv(sim_participants_file, sep='\t', header=None, skiprows=1, names=correct_columns)
sim_participants_df['Trial'] = sim_participants_df['Trial'].astype(int)
sim_participants_df['Staircase_name'] = sim_participants_df['Staircase_name'].astype(str)
sim_participants_df['Modified_Staircase_name'] = sim_participants_df['Staircase_name'].str[:-2]

# Recode trial numbers
sim_participants_df.loc[:, 'Recode_Trial'] = np.where(
    sim_participants_df['Staircase_name'].str.contains('400'),
    sim_participants_df['Trial'], 
    sim_participants_df['Trial'] + 15
)

# Get participant IDs
sim_participant_ids = sim_participants_df['Participant_ID'].unique().tolist()

# Temporary storage for current iteration's results
Pred_PSS_Accuracy = pd.DataFrame(columns=['Participant_ID', 'Pred_PSS_Value', 'Pred_Accuracy', 'RMSE', 'True_PSS_Value', 'True_Accuracy', 'True_RMSE'])
rmse_all_table = pd.DataFrame()
final_pred_pss_accuracy = pd.DataFrame()
all_mean_dfs = []

pss_index = {v: i for i, v in enumerate(pred_pss_values)}
accuracy_index = {v: i for i, v in enumerate(pred_accuracies)}

#⚠️ IF EXCLUDING PARTICIPANTS WHO HAVE RANDOM BUTTON PRESS A PARALLEL LINES
excluded_ids = set(exclude_ids_parallel) | set(exclude_ids_random)


# Loop over each participant ID
for participant_id in sim_participant_ids[1:10001]:   
    # ⚠️ Exclude if participant_id is in either exclusion set
    if participant_id in excluded_ids:
        print(f"Skipping excluded participant: {participant_id}")
        continue
        
    #print(participants_df)
    # Call your participant_matrix function (assume it's already defined elsewhere)
    pred_pss_value, pred_accuracy, rmse, rmse_table = participant_matrix(
        sim_participants_df, participant_id, pivot_df, drop=True, plots=False, verbose=False
    )
    
    #print(rmse_table)
    row = sim_participants_df[sim_participants_df["Participant_ID"] == participant_id].iloc[0]  # Take the first occurrence

    true_pss_value = row["PSS_value"]
    true_accuracy = row["Accuracy"]

    # Extract RMSE value
    rmse_true = rmse_table.loc[true_pss_value, true_accuracy]

    new_row = pd.DataFrame({
        'Participant_ID': [participant_id],
        'Pred_PSS_Value': [pred_pss_value],
        'Pred_Accuracy': [pred_accuracy],
        'RMSE': [rmse],
        'True_PSS_Value': [true_pss_value],
        'True_Accuracy': [true_accuracy],
        'True_RMSE': [rmse_true]
        })
    
    # Concatenate the new row with the existing DataFrame
    Pred_PSS_Accuracy = pd.concat([Pred_PSS_Accuracy, new_row], ignore_index=True)

    # Append results to rmse_all_table
    #rmse_table['Participant_ID'] = participant_id
    #rmse_all_table = pd.concat([rmse_all_table, rmse_table], ignore_index=False)  

# Create group structure for aligning means
group_structure = pd.DataFrame(
    [(v, a) for v in pred_pss_values for a in pred_accuracies],
    columns=['Pred_PSS_Value', 'Pred_Accuracy']
)

# Group by and calculate means
grouped_rmse_mean = Pred_PSS_Accuracy.groupby(["Pred_PSS_Value", "Pred_Accuracy"])["RMSE"].mean().reset_index()
file_mean = group_structure.merge(grouped_rmse_mean, on=['Pred_PSS_Value', 'Pred_Accuracy'], how='left')

# Pivot to create mean_df
mean_df = file_mean.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="RMSE")

# Convert mean_df to a slice of the 3D matrix
slice_matrix = np.full((len(pred_pss_values), len(pred_accuracies)), np.nan)
for pss_val, row in mean_df.iterrows():
    for acc_val, rmse in row.items():
        if not np.isnan(rmse):
            slice_matrix[pss_index[pss_val], accuracy_index[acc_val]] = rmse

# Add the slice to the list
all_mean_dfs.append(slice_matrix)
# Add list of PSS_Accuracy to mega list
final_pred_pss_accuracy = pd.concat([final_pred_pss_accuracy, Pred_PSS_Accuracy], ignore_index=True)

final_pred_pss_accuracy.to_csv("./10000_simulated_participants_100_400_pred_pss_accuracy.tsv", sep = '\t', index=False)  
np.save('./10000_simulated_participants_100_400_all_mean_rmse_plot.npy', all_mean_dfs)


# If all files have been generated run from here on:
<a id="skip-here"></a>

In [ ]:
#REFERENCE FILES:
ref_diff_df = pd.read_csv("/PATH/Reference_diff_df_100_400.csv")  

#NULL FILES
null_pred_pss_accuracy = pd.read_csv("/PATH//10000_null_simulations_100_400_pred_pss_accuracy.csv")  
null_all_mean_dfs = np.load("/PATH//10000_null_simulations_100_400_all_mean_rmse_plot.npy")
null_diff_df = pd.read_csv("/PATH//Null_diff_df_100_400.csv")  

## SIMULATED PARTICIPANT FILES:
sim_participant_pred_pss_accuracy = pd.read_csv('/PATH/10000_simulated_participants_100_400_pred_pss_accuracy_FULLRANGE.tsv', sep='\t')
sim_participant_all_mean_df = np.load("/PATH//10000_simulated_participants_100_400_all_mean_rmse_plot_FULLRANGE.npy")
sim_participant_diff_df = pd.read_csv("/PATH/10000_simulated_participants_diff_df_100_400.csv")  

# Define fixed ranges for the structure
pred_pss_values = np.arange(100, 401, 50)  # Pred_PSS_Value: 0 to 400, increments of 50
pred_accuracies = np.arange(0, 1.1, 0.1).round(1)  # Pred_Accuracy: 0.1 to 1.0, increments of 0.1


# ✅❎ Compare Predicted to True: Simulated sample

In [ ]:
# Filter out excluded participants
True_and_Pred = sim_participant_pred_pss_accuracy[~sim_participant_pred_pss_accuracy['Participant_ID'].isin(set(exclude_ids_parallel) | set(exclude_ids_random))]

# Define figure size in cm
fig_width_cm = 8
fig_height_cm = 6
fig_size_in = (fig_width_cm / 2.54, fig_height_cm / 2.54)

# Define full range of accuracy values and proportions
full_accuracy_vals = np.arange(0, 101, 10)
full_accuracy_props = full_accuracy_vals / 100

# --- True Accuracy vs PSS Value ---
grouped_true = True_and_Pred.groupby(["True_PSS_Value", "True_Accuracy"]).size().reset_index(name="Frequency")
heatmap_data_true = grouped_true.pivot(index="True_PSS_Value", columns="True_Accuracy", values="Frequency").fillna(0)
heatmap_data_true = heatmap_data_true.reindex(columns=full_accuracy_props, fill_value=0).iloc[::-1]

plt.figure(figsize=fig_size_in, dpi=300)
ax1 = sns.heatmap(
    heatmap_data_true,
    cmap=sim_participant_cmap,
    annot=True,
    fmt=".0f",
    vmin=0,
    vmax=400,
    annot_kws={"size": 5},
    cbar_kws={"label": "Frequency of sample"}
)
ax1.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax1.set_xticklabels(full_accuracy_vals, fontsize=7)
ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=7)
ax1.set_xlabel("True Accuracy (%)", fontsize=7)
ax1.set_ylabel("True PSS Value (ms)", fontsize=7)
ax1.collections[0].colorbar.ax.tick_params(labelsize=7)
plt.tight_layout()
plt.savefig("10000_simulated_participants_100_400_true_pss_accuracy.png")
plt.show()

# --- Predicted Accuracy vs Predicted PSS Value ---
grouped_pred = True_and_Pred.groupby(["Pred_PSS_Value", "Pred_Accuracy"]).size().reset_index(name="Frequency")
heatmap_data_pred = grouped_pred.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="Frequency").fillna(0)
heatmap_data_pred = heatmap_data_pred.reindex(columns=full_accuracy_props, fill_value=0).iloc[::-1]

plt.figure(figsize=fig_size_in, dpi=300)
ax2 = sns.heatmap(
    heatmap_data_pred,
    cmap=sim_participant_cmap,
    annot=True,
    fmt=".0f",
    vmin=0,
    vmax=400,
    annot_kws={"size": 5},
    cbar_kws={"label": "Frequency of sample"}
)
ax2.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax2.set_xticklabels(full_accuracy_vals, fontsize=7)
ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=7)
ax2.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax2.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax2.collections[0].colorbar.ax.tick_params(labelsize=7)
plt.tight_layout()
plt.savefig("10000_simulated_participants_100_400_pred_pss_accuracy.png")
plt.show()

# --- Mean RMSE heatmap ---
sim_participant_all_mean_df_plot = np.array(sim_participant_all_mean_df)
sim_participants_mean_across_slices = np.nanmean(sim_participant_all_mean_df_plot, axis=0)
sim_participants_mean_df_result = pd.DataFrame(
    sim_participants_mean_across_slices,
    index=pred_pss_values,
    columns=pred_accuracies
)
sim_participants_mean_df_result = sim_participants_mean_df_result.reindex(columns=full_accuracy_props, fill_value=0).iloc[::-1]

plt.figure(figsize=fig_size_in, dpi=300)
ax3 = sns.heatmap(
    sim_participants_mean_df_result,
    annot=True,
    fmt=".1f",
    cmap=sim_participant_cmap,
    vmin=0,
    annot_kws={"size": 5},
    cbar_kws={'label': 'Mean RMSE'}
)
ax3.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax3.set_xticklabels(full_accuracy_vals, fontsize=7)
ax3.set_yticklabels(ax3.get_yticklabels(), fontsize=7)
ax3.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax3.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax3.collections[0].colorbar.ax.tick_params(labelsize=7)
plt.tight_layout()
plt.savefig("10000_simulated_participants_100_400_all_mean_rmse_plot.png")
plt.show()


In [ ]:
# Extract accuracy values
true_acc_values = True_and_Pred["True_Accuracy"]
pred_acc_values = True_and_Pred["Pred_Accuracy"]
fig_width_cm = 6
fig_height_cm = 6  # adjust as needed
fig_size_in = (fig_width_cm / 2.54, fig_height_cm / 2.54)  # convert cm to inches

true_acc_values = True_and_Pred["True_Accuracy"] * 100
pred_acc_values = True_and_Pred["Pred_Accuracy"] * 100

# Create the plot
plt.figure(figsize=fig_size_in, dpi= 300)
sns.histplot(true_acc_values, bins=21, color="#ecd1b7", label="True Accuracy", kde=False, alpha=0.7)
sns.histplot(pred_acc_values, bins=21, color="#e17a4b", label="Predicted Accuracy", kde=False, alpha=0.4)

# Labels and title
plt.xlabel("Accuracy Level (%)")
plt.ylabel("Frequency")
plt.title("Distribution of True vs Predicted Accuracy")
#plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.show()


## ✅❎ 3 Thresholds of Correct 

🟩100%

🟧+/- 50ms 10%

🟥+/- 100ms 20%




___________________________
🟩100%

In [ ]:
#True_and_Pred = sim_participant_pred_pss_accuracy

## PSS  
True_and_Pred['Diff_true_pred_PSS'] = True_and_Pred['True_PSS_Value'] - True_and_Pred['Pred_PSS_Value']
zero_diff_df_pss = True_and_Pred[True_and_Pred['Diff_true_pred_PSS'] == 0]
n_pss = zero_diff_df_pss['Participant_ID'].nunique()
total_participants = True_and_Pred['Participant_ID'].nunique()
perc_pss = round((n_pss / total_participants) * 100, 2)
print(f"{n_pss} participants ({perc_pss}%) had their PSS score predicted with perfect accuracy")
print(f"\n{n_pss} participants ({perc_pss}%) had PSS correctly identified")
print(f"Mean diff between true PSS and predicted PSS: {True_and_Pred['Diff_true_pred_PSS'].mean()} +/- {True_and_Pred['Diff_true_pred_PSS'].std()}")

## ACCURACY:
True_and_Pred['Diff_true_pred'] = True_and_Pred['True_Accuracy'] -  True_and_Pred['Pred_Accuracy']
zero_diff_df_accuracy = True_and_Pred[True_and_Pred['Diff_true_pred'] == 0]
n_accuracy = zero_diff_df_accuracy['Participant_ID'].nunique()
perc_accuracy = round((n_accuracy / total_participants) * 100, 2)
print(f"{n_accuracy} participants ({perc_accuracy}%) had accuracy correctly identified")
print(f"Mean diff between true Accuracy and predicted Accuracy: {True_and_Pred['Diff_true_pred'].mean()} +/- {True_and_Pred['Diff_true_pred'].std()} \n")

def calculate_accuracy_100(row):
    if (row["Pred_PSS_Value"] == row["True_PSS_Value"]) and (row["Pred_Accuracy"] == row["True_Accuracy"]):
        return 1  # Fully accurate
    else:
        return 0  # Not accurate

True_and_Pred["Correctly_Identified"] = True_and_Pred.apply(calculate_accuracy_100, axis=1)

## PSS and ACCURACY:
correct = True_and_Pred["Correctly_Identified"].sum()
correct_perc = round((correct / total_participants) * 100, 2)
print(f"{correct} participants ({correct_perc}%) had both PSS and Accuracy correctly identified")

## Transform for plotting
# Transform for plotting
grouped = True_and_Pred.groupby(["Pred_Accuracy", "Pred_PSS_Value"]).agg(
    Correct_Percentage=("Correctly_Identified", "mean")
).reset_index()

grouped["Correct_Percentage"] *= 100
heatmap_data = grouped.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="Correct_Percentage")
heatmap_data = heatmap_data.iloc[::-1]

fig_width_cm = 8
fig_height_cm = 6
fig_size_in = (fig_width_cm / 2.54, fig_height_cm / 2.54)

plt.figure(figsize=fig_size_in, dpi=300)
ax = sns.heatmap(
    heatmap_data, 
    cmap=sim_participant_cmap, 
    vmin=0,
    vmax=100,
    annot=True,
    fmt=".1f",
    annot_kws={"size": 5},
    cbar_kws={"label": "Percentage Correctly Identified"}
)

# Define full range of accuracy values and proportions for ticks
full_accuracy_vals = np.arange(0, 101, 10)
full_accuracy_props = full_accuracy_vals / 100

ax.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax.set_xticklabels(full_accuracy_vals, fontsize=7)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=7)
ax.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax.collections[0].colorbar.ax.tick_params(labelsize=7)
plt.title("Percentage Correctly Identified 100%", fontsize=7)
plt.tight_layout()
plt.show()


🟧+/- 50ms 10%

In [ ]:
## PSS
# Calculate the absolute difference between true PSS and predicted PSS
True_and_Pred['Abs_Diff_true_pred_PSS'] = abs(
    True_and_Pred['True_PSS_Value'] - True_and_Pred['Pred_PSS_Value']
)
within_50_df = True_and_Pred[True_and_Pred['Abs_Diff_true_pred_PSS'] <= 51]
participants_within_50 = within_50_df['Participant_ID'].nunique()
total_participants = True_and_Pred['Participant_ID'].nunique()
percent_within_50 = round((participants_within_50 / total_participants) * 100, 2)
print(f"{participants_within_50} participants ({percent_within_50}%) had Pred_PSS_Value within 50ms of their true PSS value.")

## ACCURACY:
# Calculate the absolute difference between true accuracy and predicted accuracy
True_and_Pred['Abs_Diff_true_pred_accuracy'] = abs(
    True_and_Pred['True_Accuracy'] - True_and_Pred['Pred_Accuracy']
)
within_0_1_df = True_and_Pred[True_and_Pred['Abs_Diff_true_pred_accuracy'] <= 0.1]
participants_within_0_1 = within_0_1_df['Participant_ID'].nunique()
percent_within_0_1 = round((participants_within_0_1 / total_participants) * 100, 2)
print(f"{participants_within_0_1} participants ({percent_within_0_1}%) had Pred_Accuracy within 10% of their true Accuracy.")

## How many were 100% correct - Both PSS and ACCURACY:
correct = (True_and_Pred["Correctly_Identified"] != 0).sum().sum()
correct_perc = round((correct / total_participants) * 100, 2)
print(f"{correct} participants ({correct_perc}%) had both PSS and Accuracy identified within 50ms PSS and 10% accuracy.")

def calculate_accuracy_100_50(row):
    if (row["Pred_PSS_Value"] == row["True_PSS_Value"]) and (row["Pred_Accuracy"] == row["True_Accuracy"]):
        return 1  # Fully accurate
    elif (abs(row["Pred_PSS_Value"] - row["True_PSS_Value"]) <= 51) and (abs(row["Pred_Accuracy"] - row["True_Accuracy"]) <= 0.11):
        return 0.5  # Partially accurate
    else:
        return 0  # Not accurate

True_and_Pred["Correctly_Identified"] = True_and_Pred.apply(calculate_accuracy_100_50, axis=1)

grouped = True_and_Pred.groupby(["Pred_Accuracy", "Pred_PSS_Value"]).agg(
    Correct_Percentage=("Correctly_Identified", "mean")
).reset_index()

## How many were 100% correct - Both PSS and ACCURACY:
correct = (True_and_Pred["Correctly_Identified"] != 0).sum().sum()
correct_perc = round((correct / total_participants) * 100, 2)
print(f"{correct} participants ({correct_perc}%) had both PSS and Accuracy identified within 50ms PSS and 10% accuracy.")

display(HTML(True_and_Pred.head().to_html()))
print(f"{correct} participants correctly identified within 50ms PSS and 10% accuracy")

grouped["Correct_Percentage"] *= 100

heatmap_data = grouped.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="Correct_Percentage")
heatmap_data = heatmap_data.iloc[::-1]

fig_width_cm = 8
fig_height_cm = 6
fig_size_in = (fig_width_cm / 2.54, fig_height_cm / 2.54)

plt.figure(figsize=fig_size_in, dpi=300)
ax = sns.heatmap(
    heatmap_data, 
    cmap=sim_participant_cmap, 
    vmin=0,
    vmax=100,
    annot=True,
    fmt=".1f",
    annot_kws={"size": 5},
    cbar_kws={"label": "Percentage Correctly Identified"}
)

# Define full range of accuracy values and proportions for ticks
full_accuracy_vals = np.arange(0, 101, 10)
full_accuracy_props = full_accuracy_vals / 100

ax.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax.set_xticklabels(full_accuracy_vals, fontsize=7)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=7)
ax.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax.collections[0].colorbar.ax.tick_params(labelsize=7)

plt.title("within 50ms PSS and 10% Accuracy", fontsize=7)
plt.tight_layout()
plt.show()

🟥+/- 100ms 20%

In [ ]:
## How many were 100% +/- 100ms PSS and 20% correct:

## PSS
within_100_df = True_and_Pred[True_and_Pred['Abs_Diff_true_pred_PSS'] <= 101]
participants_within_100 = within_100_df['Participant_ID'].nunique()
percent_within_100 = round((participants_within_100 / total_participants) * 100, 2)
print(f"{participants_within_100} participants ({percent_within_100}%) had Pred_PSS_Value within 100ms of their true PSS value.")

## ACCURACY
within_0_2_df = True_and_Pred[True_and_Pred['Abs_Diff_true_pred_accuracy'] <= 0.2]
participants_within_0_2 = within_0_2_df['Participant_ID'].nunique()
percent_within_0_2 = round((participants_within_0_2 / total_participants) * 100, 2)
print(f"{participants_within_0_2} participants ({percent_within_0_2}%) had Pred_Accuracy within 20% of their true Accuracy.")

def calculate_accuracy_100_50_25(row):
    if (row["Pred_PSS_Value"] == row["True_PSS_Value"]) and (row["Pred_Accuracy"] == row["True_Accuracy"]):
        return 1  # Fully accurate
    elif (abs(row["Pred_PSS_Value"] - row["True_PSS_Value"]) <= 51) and (abs(row["Pred_Accuracy"] - row["True_Accuracy"]) <= 0.11):
        return 0.5  # Partially accurate
    elif (abs(row["Pred_PSS_Value"] - row["True_PSS_Value"]) <= 101) and (abs(row["Pred_Accuracy"] - row["True_Accuracy"]) <= 0.21):
        return 0.25  # Somewhat accurate
    else:
        return 0  # Not accurate

# Apply classification
True_and_Pred["Correctly_Identified"] = True_and_Pred.apply(calculate_accuracy_100_50_25, axis=1)
display(HTML(True_and_Pred.head().to_html()))

correct = (True_and_Pred["Correctly_Identified"] != 0).sum().sum()
correct_perc = round((correct / True_and_Pred['Participant_ID'].nunique()) * 100, 2)
print(f"{correct} participants ({correct_perc}%) had both PSS and Accuracy correctly identified within 100ms PSS and 20% Accuracy.")

# Grouped accuracy for heatmap
grouped = True_and_Pred.groupby(["Pred_Accuracy", "Pred_PSS_Value"]).agg(
    Correct_Percentage=("Correctly_Identified", "mean")
).reset_index()

grouped["Correct_Percentage"] *= 100

heatmap_data = grouped.pivot(index="Pred_PSS_Value", columns="Pred_Accuracy", values="Correct_Percentage")
heatmap_data = heatmap_data.iloc[::-1]

fig_width_cm = 8
fig_height_cm = 6
fig_size_in = (fig_width_cm / 2.54, fig_height_cm / 2.54)

plt.figure(figsize=fig_size_in, dpi=300)
ax = sns.heatmap(
    heatmap_data, 
    cmap=sim_participant_cmap, 
    vmin=0,
    vmax=100,
    annot=True,
    fmt=".1f",
    annot_kws={"size": 5},
    cbar_kws={"label": "Percentage Correctly Identified"}
)

full_accuracy_vals = np.arange(0, 101, 10)
ax.set_xticks(np.arange(len(full_accuracy_vals)) + 0.5)
ax.set_xticklabels(full_accuracy_vals, fontsize=7)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=7)
ax.set_xlabel("Predicted Accuracy (%)", fontsize=7)
ax.set_ylabel("Predicted PSS Value (ms)", fontsize=7)
ax.collections[0].colorbar.ax.tick_params(labelsize=7)

plt.title("within 100ms PSS and 20% Accuracy", fontsize=7)
plt.tight_layout()
plt.show()


## SIM PARTICIPANTS: Real vs Predicted PSS and of Accuracy

In [ ]:
plt.figure(figsize=(20, 12))

# Plot 1: Predicted vs Real PSS
plt.subplot(2, 2, 1)

# Group by the PSS values and count the occurrences
pss_counts = True_and_Pred.groupby(['True_PSS_Value', 'Pred_PSS_Value']).size().reset_index(name='Count')
merged_data_with_pss_counts = pd.merge(True_and_Pred, pss_counts, on=['True_PSS_Value', 'Pred_PSS_Value'], how='left')

# Calculate the number of unique participants
unique_participants = True_and_Pred['Participant_ID'].nunique()
merged_data_with_pss_counts['Percent_of_sample'] = (merged_data_with_pss_counts['Count'] / unique_participants) * 100

norm_pss = plt.Normalize(vmin=merged_data_with_pss_counts['Percent_of_sample'].min(), vmax=merged_data_with_pss_counts['Percent_of_sample'].max())

# Scatter plot with color mapping based on the count of participants
scatter_pss = plt.scatter(
    merged_data_with_pss_counts['True_PSS_Value'], 
    merged_data_with_pss_counts['Pred_PSS_Value'], 
    c=merged_data_with_pss_counts['Percent_of_sample'], 
    cmap='Reds', alpha=0.6, s=100, norm=norm_pss
)

plt.plot([0, 600], [0, 600], 'r--', alpha = 0.3)
plt.title('Predicted PSS vs Real PSS')
plt.xlabel('Real PSS values')
plt.ylabel('Predicted PSS values')
plt.colorbar(scatter_pss, label='% of sample')



# Plot 2: Real Accuracy vs Predicted Accuracy
plt.subplot(2, 2, 2)

# Group by Accuracy and count the occurrences
accuracy_counts = True_and_Pred.groupby(['True_Accuracy', 'Pred_Accuracy']).size().reset_index(name='Count')
merged_data_with_accuracy_counts = pd.merge(True_and_Pred, accuracy_counts, on=['True_Accuracy', 'Pred_Accuracy'], how='left')
merged_data_with_accuracy_counts['Percent_of_sample'] = (merged_data_with_accuracy_counts['Count'] / unique_participants) * 100

# Normalize the count values for color mapping
norm_accuracy = plt.Normalize(vmin=merged_data_with_accuracy_counts['Percent_of_sample'].min(), vmax=merged_data_with_accuracy_counts['Percent_of_sample'].max())

# Scatter plot with color mapping based on the count of participants
scatter_accuracy = plt.scatter(
    merged_data_with_accuracy_counts['True_Accuracy']*100, 
    merged_data_with_accuracy_counts['Pred_Accuracy']*100, 
    c=merged_data_with_accuracy_counts['Percent_of_sample'], 
    cmap='Reds', alpha=0.6, s=100, norm=norm_accuracy
)

plt.plot([0, 100], [0, 100], 'r--',alpha = 0.3)
plt.title('Real Accuracy vs Predicted Accuracy')
plt.xlabel('Real Accuracy (%)')
plt.ylabel('Predicted Accuracy (%)')
plt.colorbar(scatter_accuracy, label='% of sample')



#Plot 3: CORRECT PSS: Real Accuracy vs Predicted Accuracy
plt.subplot(2, 2, 4)

# Initialize a list to store filtered data
filtered_data = []
for participant_id in True_and_Pred['Participant_ID'].unique():
    matching_rows = True_and_Pred[True_and_Pred['Participant_ID'] == participant_id]
    
    # Check if Pred_PSS_Value matches PSS_value
    participant_PSS = matching_rows['True_PSS_Value'].iloc[0]
    matching_rows = matching_rows[matching_rows['Pred_PSS_Value'] == participant_PSS]
    
    if len(matching_rows) == 1:
        row = matching_rows.iloc[0]
        
        # Get predicted accuracy (minimum value among accuracy columns)
        pred_accuracy = row['Pred_Accuracy']
        real_accuracy = row['True_Accuracy']
        
        # Store relevant data
        filtered_data.append({
            'Participant_ID': row['Participant_ID'],
            'PSS_value_real': row['True_PSS_Value'],
            'Real_Accuracy': real_accuracy,
            'Pred_Accuracy': pred_accuracy
        })

# Convert list to DataFrame
filtered_data_PSS = pd.DataFrame(filtered_data)

# Count occurrences of each (Real_Accuracy, Pred_Accuracy) combination
accuracy_counts = (
    filtered_data_PSS.groupby(['Real_Accuracy', 'Pred_Accuracy'])
    .size()
    .reset_index(name='Count')
)

# Normalize frequency for color mapping
accuracy_counts['Frequency'] = (accuracy_counts['Count'] / len(filtered_data_PSS['Participant_ID'].unique())) * 100
norm = plt.Normalize(vmin=accuracy_counts['Frequency'].min(), vmax=accuracy_counts['Frequency'].max())

# Create scatter plot

scatter = plt.scatter(
    accuracy_counts['Real_Accuracy'] * 100, 
    accuracy_counts['Pred_Accuracy'] * 100, 
    c=accuracy_counts['Frequency'], 
    cmap='OrRd', alpha=0.6, s=100, norm=norm
)

# Add reference line
plt.plot([0, 100], [0, 100], 'r--', alpha=0.3)

# Labels and title
plt.title('Correct PSS: Real Accuracy vs Predicted Accuracy')
plt.xlabel('Real Accuracy Level (%)')
plt.ylabel('Predicted Accuracy (%)')

# Color bar
plt.colorbar(scatter, label='% of sample')

# Show plot
plt.tight_layout()



#Plot 4: CORRECT ACCURACY: Real PSS vs Predicted PSS
plt.subplot(2, 2, 3)

# Initialize a list to store filtered data
filtered_data = []
for participant_id in True_and_Pred['Participant_ID'].unique():
    matching_rows = True_and_Pred[True_and_Pred['Participant_ID'] == participant_id]
    
    # Check if Pred_PSS_Value matches PSS_value
    participant_Accuracy = matching_rows['True_Accuracy'].iloc[0]
    matching_rows = matching_rows[matching_rows['Pred_Accuracy'] == participant_Accuracy]
    
    if len(matching_rows) == 1:
        row = matching_rows.iloc[0]
        
        # Get predicted accuracy (minimum value among accuracy columns)
        pred_PSS = row['Pred_PSS_Value']
        real_PSS = row['True_PSS_Value']
        
        # Store relevant data
        filtered_data.append({
            'Participant_ID': row['Participant_ID'],
            'PSS_value_real': row['True_PSS_Value'],
            'Real_PSS_value': pred_PSS,
            'Pred_PSS_Value': real_PSS
        })

# Convert list to DataFrame
filtered_data_Accuracy = pd.DataFrame(filtered_data)

# Count occurrences of each (Real_Accuracy, Pred_Accuracy) combination
PSS_counts = (
    filtered_data_Accuracy.groupby(['Real_PSS_value', 'Pred_PSS_Value'])
    .size()
    .reset_index(name='Count')
)

# Normalize frequency for color mapping
PSS_counts['Frequency'] = (PSS_counts['Count'] / len(filtered_data_Accuracy['Participant_ID'].unique())) * 100
norm = plt.Normalize(vmin=PSS_counts['Frequency'].min(), vmax=PSS_counts['Frequency'].max())

# Create scatter plot

scatter = plt.scatter(
    PSS_counts['Pred_PSS_Value'], 
    PSS_counts['Pred_PSS_Value'], 
    c=PSS_counts['Frequency'], 
    cmap='OrRd', alpha=0.6, s=100, norm=norm
)

# Add reference line
plt.plot([0, 600], [0, 600], 'r--', alpha=0.3)

# Labels and title
plt.title('Correct Accuracy: Real PSS vs Predicted PSS')
plt.xlabel('Real PSS')
plt.ylabel('Predicted PSS')

# Color bar
plt.colorbar(scatter, label='% of sample')

# Show plot
plt.tight_layout()
plt.show()


## SIM PARTICIPANTS:
Get P-value for where they fall in the NULL distrubtion 

In [ ]:
#REFERENCE FILES:
ref_diff_df = pd.read_csv("/PATH/Reference_diff_df_100_400.csv")  

#NULL FILES
null_pred_pss_accuracy = pd.read_csv("/PATH/10000_null_simulations_100_400_pred_pss_accuracy.csv")  
null_all_mean_dfs = np.load("/PATH/10000_null_simulations_100_400_all_mean_rmse_plot.npy")
null_diff_df = pd.read_csv("/PATH/Null_diff_df_100_400.csv")  

## SIMULATED PARTICIPANT FILES:
sim_participant_pred_pss_accuracy = pd.read_csv('/PATH/10000_simulated_participants_100_400_pred_pss_accuracy.tsv', sep='\t')
sim_participant_all_mean_df = np.load("/PATH/10000_simulated_participants_100_400_all_mean_rmse_plot.npy")
sim_participant_diff_df = pd.read_csv("/PATH/10000_simulated_participants_diff_df_100_400.csv")  


In [ ]:
from scipy.stats import norm, linregress
#If plotting - shows disribution and participant as line
plotting = False

sim_participants_file = f'/PATH/10000_simulated_participants_100_400.tsv'
sim_participants_df = pd.read_csv(sim_participants_file, sep='\t')

sim_participant_pred_pss_accuracy = compute_p_values(sim_participant_pred_pss_accuracy, df_vector_across_slices, plotting=True)
#Saves p-values into the Pred_PSS_Accuracy_drop_real file
sim_participant_pred_pss_accuracy.to_csv('/PATH/10000_simulated_participants_100_400_pred_pss_accuracy_pvals.tsv', sep='\t', index=False)

## SIM PARTICIPANTS:
Disrbution of P-values

In [ ]:
# Assuming p_values is defined
p_values = sim_participant_pred_pss_accuracy['p_value']

plt.figure(figsize=(8, 6))
sns.histplot(p_values, bins=100, kde=True, color='#e86a36', alpha=0.6)
plt.title('Distribution of P-values')
plt.xlabel('P-value')
plt.ylabel('Frequency')

ax_inset = inset_axes(plt.gca(), width="60%", height="40%", loc="upper right")

# Plot the zoomed-in region
sns.histplot(p_values, bins=100, kde=False, color='#e86a36', alpha=0.6, ax=ax_inset)
ax_inset.set_xlim(0, 0.08)
ax_inset.set_ylim(0, None)  # Adjust if needed
ax_inset.set_xticks([0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07])
ax_inset.xlabel('P-value')
ax_inset.ylabel('Frequency')
ax_inset.set_yticks([]) 
plt.show()


## Mean RMSE across participants, sig vs non sig

In [ ]:
significant_participants = []
non_significant_participants  = []
# Separate participants into two groups based on p-value
significant_participants = sim_participant_pred_pss_accuracy[sim_participant_pred_pss_accuracy['p_value'] < 0.05]
print(len(significant_participants))
non_significant_participants = sim_participant_pred_pss_accuracy[sim_participant_pred_pss_accuracy['p_value'] >= 0.05]
print(len(non_significant_participants))

# Calculate the mean RMSE for each group
mean_rmse_significant = significant_participants.groupby(['Pred_PSS_Value', 'Pred_Accuracy'])['RMSE'].mean().unstack()
mean_rmse_significant = mean_rmse_significant.iloc[::-1]

mean_rmse_non_significant = non_significant_participants.groupby(['Pred_PSS_Value', 'Pred_Accuracy'])['RMSE'].mean().unstack()
mean_rmse_non_significant = mean_rmse_non_significant.iloc[::-1]

# Plot the heatmap for participants with p < 0.05 (significant)
plt.figure(figsize=(12, 8))
sns.heatmap(mean_rmse_significant, cmap=sim_participant_cmap, annot=True, fmt='.2f', cbar_kws={'label': 'Mean RMSE'}, vmin = 0, vmax = 30)
plt.title('Heatmap of Mean RMSE for Participants with p < 0.05')
plt.xlabel('Pred Accuracy')
plt.ylabel('Pred PSS Value')
plt.show()


# Plot the heatmap for participants with p >= 0.05 (non-significant)
plt.figure(figsize=(12, 8))
sns.heatmap(mean_rmse_non_significant, cmap=sim_participant_cmap, annot=True, fmt='.2f', cbar_kws={'label': 'Mean RMSE'}, vmin = 0, vmax = 30)
plt.title('Heatmap of Mean RMSE for Participants with p >= 0.05')
plt.xlabel('Pred Accuracy')
plt.ylabel('Pred PSS Value')
plt.show()


In [ ]:
significant_participants = updated_participant_data[updated_participant_data['p_value'] < 0.05]
non_significant_participants = updated_participant_data[updated_participant_data['p_value'] >= 0.05]

# Initialize an empty list to store the result
vector_across_slices = []

# Loop over each (y, x) position (same logic as in the original code)
for y in range(all_mean_dfs_plot.shape[1]):  # 12 rows
    row = []
    for x in range(all_mean_dfs_plot.shape[2]):  # 11 columns
        values_at_pos = all_mean_dfs_plot[:, y, x].tolist()
        row.append(values_at_pos)
    vector_across_slices.append(row)

df_vector_across_slices = pd.DataFrame(vector_across_slices)
df_vector_across_slices.index = pred_pss_values  # Assuming `pred_pss_values` is the list of index labels
df_vector_across_slices.columns = pred_accuracies  # Assuming `pred_accuracies` is the list of column labels

# Set the number of rows and columns for the grid of subplots
num_rows = df_vector_across_slices.shape[0]
num_cols = df_vector_across_slices.shape[1]

# Calculate the min/max values for the x and y axis ranges
min_xvalue = 0
max_xvalue = math.ceil(np.nanmax([item for sublist in df_vector_across_slices.values for item in sublist]) + 0.05)
max_yvalue = 0

# Find max frequency for scaling y-axis
for cell_row in range(num_rows):
    for cell_col in range(num_cols):
        cell_values = df_vector_across_slices.iloc[cell_row, cell_col]
        hist, bin_edges = np.histogram(cell_values, bins=50, range=(min_xvalue, max_xvalue))
        max_yvalue = max(max_yvalue, hist.max())

# Create figure and axes for subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15,10))

# Loop through each subplot to create histograms
for cell_row in range(num_rows):
    for cell_col in range(num_cols):
        cell_values = df_vector_across_slices.iloc[cell_row, cell_col]
        ax = axes[num_rows - 1 - cell_row, cell_col]  # Flip the row order

        # Plot the histogram for the null distribution
        sns.histplot(cell_values, bins=50, kde=True, color='#99c3dd', edgecolor='#99c3dd', alpha=0.7, ax=ax, 
                     binrange=(min_xvalue, max_xvalue))

        # Overlay the RMSE values for significant and non-significant participants
        significant_rmse_values = significant_participants[
            (significant_participants['Pred_PSS_Value'] == pred_pss_values[cell_row]) &
            (significant_participants['Pred_Accuracy'] == pred_accuracies[cell_col])
        ]['RMSE']
        non_significant_rmse_values = non_significant_participants[
            (non_significant_participants['Pred_PSS_Value'] == pred_pss_values[cell_row]) &
            (non_significant_participants['Pred_Accuracy'] == pred_accuracies[cell_col])
        ]['RMSE']

        # Plot vertical lines for significant RMSE values in red
        for rmse_value in significant_rmse_values:
            ax.vlines(rmse_value, 0, max_yvalue, color='#ff6600', alpha=0.1, linewidth=1.5, label='Significant')

        # Plot vertical lines for non-significant RMSE values in blue
        for rmse_value in non_significant_rmse_values:
            ax.vlines(rmse_value, 0, max_yvalue, color='#ff6600', alpha=0.1, linewidth=1.5, linestyles= 'dashed',label='Non-Significant')

        # Remove axis labels and ticks for a clean look
        ax.set_xticks([]) 
        ax.set_yticks([]) 
        ax.set_ylabel('')

        # Set axis limits
        ax.set_xlim(min_xvalue, max_xvalue)
        ax.set_ylim(0, max_yvalue)

        # Clean grid appearance
        if cell_col == num_cols - 1:
            ax.spines['top'].set_visible(True)
            ax.spines['right'].set_visible(True)
            ax.spines['left'].set_visible(True)
            ax.spines['bottom'].set_visible(True)
        else:
            ax.spines['top'].set_visible(True)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(True)
            ax.spines['bottom'].set_visible(True)

        if cell_row == 0 and cell_col == 0:
            ax.set_xticks(np.linspace(min_xvalue, max_xvalue, num=2))
        else:
            ax.set_xticks([])

        if cell_col == 0 and cell_row == 0:
            ax.set_yticks(np.linspace(0, max_yvalue, num=2))

# Left Y-axis (Pred_PSS_Value)
ax_left = fig.add_axes([0.065, 0.1, 0.01, 0.75])
ax_left.set_xticks([])  
ax_left.tick_params(axis="y", direction="out", length=12, width=1.2)
ax_left.set_yticks(np.arange(num_rows) + 0.5)
ax_left.set_yticklabels(pred_pss_values, fontsize=10, ha='right')
ax_left.set_ylabel("Predicted PSS Value\nNumber of instances", fontsize=12, labelpad=20)
ax_left.tick_params(left=False, labelleft=True, right=False, labelright=False)
ax_left.spines['top'].set_visible(False)
ax_left.spines['bottom'].set_visible(False)
ax_left.spines['left'].set_visible(False)
ax_left.spines['right'].set_visible(False)

# Bottom X-axis (Pred_Accuracy)
ax_bottom = fig.add_axes([0.16, 0.06, 0.7, 0.01])
ax_bottom.set_yticks([])  
ax_bottom.set_xticks(np.arange(num_cols))
ax_bottom.set_xticklabels(pred_accuracies, rotation=0, fontsize=10, ha='center')
ax_bottom.set_xlabel("RMSE\nPredicted Accuracy", fontsize=12, labelpad=15)
ax_bottom.tick_params(axis="x", direction="inout", length=6, width=1.2)
ax_bottom.spines['top'].set_visible(False)
ax_bottom.spines['bottom'].set_visible(False)
ax_bottom.spines['left'].set_visible(False)
ax_bottom.spines['right'].set_visible(False)

# Adjust layout to fit the grid tightly
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("overlayed_vertical_line_distribution_plots.png")
plt.show()

### Probe the difference in RMSE between the predicted and true values

In [ ]:
real_vs_pred_rmse = pd.read_csv('/PATH/1000_participant_simulations_100_400_pred_pss_accuracy_withtrueRMSE_pvals.tsv', sep = '\t')
real_vs_pred_rmse['Diff_rmse'] = real_vs_pred_rmse['True_RMSE'] -real_vs_pred_rmse['RMSE']
display(HTML(real_vs_pred_rmse.head().to_html()))

In [ ]:
# Convert Accuracy to categorical explicitly
real_vs_pred_rmse['Accuracy'] = real_vs_pred_rmse['Accuracy'].astype(str)  # Convert to string to force categorical behavior

# Set plot size
plt.figure(figsize=(20, 8))

# Create a box plot for Diff_rmse
#sns.boxplot(y=real_vs_pred_rmse['Diff_rmse'], color='lightgray', width=0.3)

# Overlay scatter plot
sns.stripplot(y=real_vs_pred_rmse['Diff_rmse'], 
              x=[0] * len(real_vs_pred_rmse),  # Fix x-axis issue
              hue=real_vs_pred_rmse['Accuracy'], 
              size=real_vs_pred_rmse['PSS_value'] / 25,  # Adjust size
              palette='tab10',  # Better color variety
              jitter=0.4, 
              alpha=0.7)

# Labels and title
plt.ylabel('Difference in RMSE (Diff_rmse)')
plt.xlabel('')
plt.title('Box Plot of Diff_rmse with Scatter Points')

# Adjust legend
plt.legend(title='Accuracy', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.show()

In [ ]:
real_vs_pred_rmse['Diff_rmse'] = real_vs_pred_rmse['True_RMSE'] -real_vs_pred_rmse['RMSE']
#print(real_vs_pred_rmse)
# Set plot size
plt.figure(figsize=(10, 8))

# Create a box plot for Diff_rmse
sns.boxplot(y=real_vs_pred_rmse['Diff_rmse'], color='lightgray', width=0.3)

# Overlay scatter plot
sns.stripplot(y=real_vs_pred_rmse['Diff_rmse'], 
              x=[0] * len(real_vs_pred_rmse),  # Fix x-axis issue
              hue=real_vs_pred_rmse['PSS_value'], 
              size = 15,
              palette='tab20',  # Better color variety
              jitter=0.4, 
              alpha=0.7)

# Labels and title
plt.ylabel('Difference in RMSE (Diff_rmse)')
plt.xlabel('')
plt.title('Box Plot of Diff_rmse with Scatter Points')

# Adjust legend
plt.legend(title='PSS value', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.show()

In [ ]:
# Ensure Accuracy is categorical (convert to string for proper grouping)

# Ensure Accuracy is categorical (convert to string for proper grouping)
real_vs_pred_rmse['Accuracy'] = real_vs_pred_rmse['Accuracy'].astype(str)

# Set plot size
plt.figure(figsize=(10, 6))

# Identify Accuracy categories that have variance 
unique_accuracy = real_vs_pred_rmse['Accuracy'].unique()

# Create a KDE plot for each Accuracy level, excluding those with no variance
for accuracy in unique_accuracy:
    subset = real_vs_pred_rmse[real_vs_pred_rmse['Accuracy'] == accuracy]
    
    # Check for variance
    if subset['Diff_rmse'].var() > 0:  # If there is variance, plot it
        sns.kdeplot(data=subset, 
                    x="Diff_rmse", 
                    label=accuracy, 
                    fill=False,
                    common_norm=False,  # Don't normalize across categories
                    palette="tab10",  # Use a distinct palette
                    alpha=0.1)  # Transparency for better visibility

# Labels and title
plt.xlabel("Difference in RMSE (Diff_rmse)")
plt.ylabel("Density")
plt.title("Density Plot of Difference Values per Accuracy")

# Show legend
plt.legend(title="Accuracy", bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.show()

In [ ]:
# Ensure Accuracy is categorical (convert to string for proper grouping)

# Ensure Accuracy is categorical (convert to string for proper grouping)
real_vs_pred_rmse['Accuracy'] = real_vs_pred_rmse['Accuracy'].astype(str)

# Set plot size
plt.figure(figsize=(10, 6))

# Identify Accuracy categories that have variance 
unique_accuracy = real_vs_pred_rmse['PSS_value'].unique()

# Create a KDE plot for each Accuracy level, excluding those with no variance
for accuracy in unique_accuracy:
    subset = real_vs_pred_rmse[real_vs_pred_rmse['PSS_value'] == accuracy]
    
    # Check for variance
    if subset['Diff_rmse'].var() > 0:  # If there is variance, plot it
        sns.kdeplot(data=subset, 
                    x="Diff_rmse", 
                    label=accuracy, 
                    fill=False,
                    common_norm=False,  # Don't normalize across categories
                    palette="tab20",  # Use a distinct palette
                    alpha=0.1)  # Transparency for better visibility

# Labels and title
plt.xlabel("Difference in RMSE (Diff_rmse)")
plt.ylabel("Density")
plt.title("Density Plot of Difference Values per PSS")

# Show legend
plt.legend(title="PSS value", bbox_to_anchor=(1.05, 1), loc='upper left')

# Show plot
plt.show()

## Plot simulated participant trajectories

In [ ]:

pss_value_list = list(range(50, 601, 50))  # [50, 100, 150, ..., 600]
accuracy_list = [round(i * 0.1, 1) for i in range(0, 11)]  # [0.0, 0.1, 0.2, ..., 1.0]
n_iterations = 100

participants_file = f'/PATH/1000_participant_simulations_100_400.tsv'
participants_df = pd.read_csv(participants_file, sep='\t')
participants_df['Trial'] = participants_df['Trial'].astype(int)
participants_df['Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]

# Recode trial numbers
participants_df.loc[:, 'Recode_Trial'] = np.where(
    participants_df['Staircase_name'].str.contains('400'),
    participants_df['Trial'], 
    participants_df['Trial'] + 15
)
print(participants_df)

# Load simulation data
simulation_df = pd.read_csv('/PATH/Reference_simulations.tsv', sep='\t')

# Clean column names (remove tabs, strip spaces)
simulation_df.columns = [col.strip().replace('\t', '') for col in simulation_df.columns]

simulation_df['Staircase_name'] = simulation_df['Staircase_name'].astype(str)

# Get participant IDs
participant_ids = participants_df['Participant_ID'].unique().tolist()

# Loop over each participant ID
for participant_id in participant_ids[639:640]:
    participant_id = participant_id  # Extract the numeric part (e.g., '005')
    print(participant_id)
    # Read the participant's data file
    participants_df.columns = [col.strip().replace('\t', '') for col in participants_df.columns]

    participants_df = participants_df[participants_df['Staircase_name'] != 'Training']
    participants_df = participants_df[participants_df['Staircase_name'] != 'Post_task_question']
    participants_df['Modified_Staircase_name'] = participants_df['Staircase_name'].str[:-2]
    participants_df['Trial'] = participants_df['Trial'].astype(int)
    

    plot_simulations_with_participant_data(simulation_df, participants_df, participant_id, pss_value_list, accuracy_list, n_iterations)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Assuming p_values is defined
p_values = updated_participant_data['p_value']

# Create the main histogram
plt.figure(figsize=(8, 6))
sns.histplot(p_values, bins=100, kde=True, color='#4C72B0', alpha=0.6)

# Set plot labels and title
plt.title('Distribution of P-values')
plt.xlabel('P-value')
plt.ylabel('Frequency')

# Create inset axes
ax_inset = inset_axes(plt.gca(), width="60%", height="40%", loc="center")

# Plot the zoomed-in region
sns.histplot(p_values, bins=100, kde=False, color='#4C72B0', alpha=0.6, ax=ax_inset)
ax_inset.set_xlim(0, 0.08)
ax_inset.set_ylim(0, None) 
ax_inset.set_xticks([0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07])
#ax_inset.xlabel('P-value')
#ax_inset.ylabel('Frequency')

ax_inset.set_yticks([])  

# Show plot
plt.show()


## Get probability distribution for HDT: 40 x 0.5


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

def simulate_trials(n_simulations=10000, n_trials=40, p_correct=0.5):
    """Simulates random guessing over multiple trials."""
    results = np.random.binomial(n_trials, p_correct, n_simulations)
    return results

def calculate_probabilities(sim_results, percentages, n_trials=40, p_correct=0.5):
    """Calculates the two-tailed probability of achieving a given percentage or more extreme."""
    # The mean and standard deviation for the binomial distribution (normal approximation)
    mean = n_trials * p_correct
    std_dev = np.sqrt(n_trials * p_correct * (1 - p_correct))
    
    # Convert percentages to the number of correct answers
    thresholds = [int(p * n_trials) for p in percentages]
    
    probabilities = []
    for threshold in thresholds:
        # Calculate z-scores for the lower and upper tails
        z_lower = (threshold - mean) / std_dev
        z_upper = (n_trials - threshold - mean) / std_dev
        
        # Find the one-tailed p-value for each tail using the normal distribution
        p_lower = norm.cdf(z_lower)
        p_upper = norm.cdf(z_upper)
        
        # Two-tailed p-value: sum the two one-tailed p-values, considering symmetry
        p_value = 2 * min(p_lower, p_upper)
        
        # Append p-value rounded to 4 decimal places
        probabilities.append(round(p_value, 4))
    
    # Prepare the result as a DataFrame
    df = pd.DataFrame({
        'Percentage': [f"{int(p * 100)}%" for p in percentages],
        'Two-Tailed P-Value': probabilities
    })
    
    return df, thresholds

# Run the simulation
sim_results = simulate_trials()

# Example: Get probabilities for several percentages
#percentages_to_check = [i / 100 for i in range(, 40)]
percentages_to_check = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # 10%, 20%, ..., 90%
df_probabilities, thresholds = calculate_probabilities(sim_results, percentages_to_check)

# Display the results as a DataFrame
print(df_probabilities)

# Plot density plot of simulation results
sns.kdeplot(sim_results, fill=True)
plt.xlabel("Number of Correct Answers")
plt.ylabel("Density")
plt.title("Density Plot of Simulation Results")

# Add vertical red lines for each percentage threshold
for threshold in thresholds:
    plt.axvline(threshold, color='red', linestyle='--')

#plt.show()

### Frequency of button presses for HDT

In [ ]:
# Simulation parameters
num_participants = 10000
num_presses = 40

# Simulate random button presses (0 for B1, 1 for B2)
button_presses = np.random.choice([0, 1], size=(num_participants, num_presses))

# Count occurrences of B1 (0) and B2 (1) per participant
b1_counts = np.sum(button_presses == 0, axis=1)
b2_counts = np.sum(button_presses == 1, axis=1)

# Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(b1_counts, bins=range(0, num_presses+2), alpha=0.6, label='B1', color='blue', edgecolor='black')
plt.hist(b2_counts, bins=range(0, num_presses+2), alpha=0.6, label='B2', color='red', edgecolor='black')
plt.xlabel("Number of presses")
plt.ylabel("Frequency of participants")
plt.title("Histogram of Button Presses per Participant")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## 14 (lower) and 26 (upper)

# I think I can delete this... but just in case

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 12))  # 3 rows, 2 columns
axes = axes.flatten()
numeric_columns = pivoted_df_with_null.select_dtypes(include=['number']).columns
num_plots = min(len(numeric_columns), len(axes))
for i in range(num_plots):
    col = numeric_columns[i]
    ax = axes[i]  # Get the corresponding subplot
    sns.kdeplot(pivoted_df_with_null[col], ax=ax, fill=True, color='steelblue', alpha=0.6)
    ax.set_title(f'KDE of {col}')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0,30)
plt.tight_layout()
plt.show()




# Filter the DataFrame for '100' responses
pivoted_100_df = pivoted_df_with_null.filter(like='100', axis=1)

custom_colors = [ '#e49971', '#ecd1b7','#e17a4b']
custom_colors = [ '#87afe6', '#b1d8ec','#6991e1']

# Plot for all '100' responses
pivoted_100_df.plot(kind='bar', stacked=True, color = custom_colors, figsize=(20, 6))
plt.xlabel("Participant")
plt.ylabel("Sum of Responses")
plt.title("Summed '100' Response Percentages by Participant")
plt.legend(title="Response Type")
plt.show()

# Filter the DataFrame for '400' responses
pivoted_400_df = pivoted_df_with_null.filter(like='400', axis=1)

# Plot for all '400' responses
pivoted_400_df.plot(kind='bar', stacked=True, color = custom_colors, figsize=(20, 6))
plt.xlabel("Participant")
plt.ylabel("Sum of Responses")
plt.title("Summed '400' Response Percentages by Participant")
plt.legend(title="Response Type")
plt.show()

